# Compute DF6 regional averages for the satellite plots

In [1]:
import regionmask
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import json
import glob

### Import the CliMAF library to do the pretreatments (https://climaf.readthedocs.io/en/master/)

In [2]:
from climaf.api import *

CliMAF version = 1.2.13


CliMAF install => /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post
python => /modfs/modtools/miniconda2/envs/analyse_2.7_v2/bin/python
---
Required softwares to run CliMAF => you are using the following versions/installations:
ncl 6.6.2 => /modfs/modtools/miniconda2/envs/analyse_2.7_v2/bin/ncl
cdo 1.9.6 => /opt/nco/1.9/bin/cdo
nco (ncks) 4.5.2 => /opt/nco-4.5.2/bin/ncks
ncdump fichier => /modfs/modtools/miniconda2/envs/analyse_2.7_v2/bin/ncdump
Check stamping requirements
nco (ncatted) found -> /opt/nco-4.5.2/bin/ncatted
convert found -> /usr/bin/convert
pdftk found -> /usr/bin/pdftk
exiv2 found -> /ciclad-home/jservon/Evaluation/CliMAF/climaf_installs/climaf_V1.2.13_post/bin/exiv2
---


Cache directory set to : /data/jservon/climafcache (use $CLIMAF_CACHE if set) 
Cache directory for remote data set to : /data/jservon/climafcache/remote_data (use $CLIMAF_REMOTE_CACHE if set) 
Available macros read from ~/.climaf.macros are : []


In [3]:
# -- Function to split a multi-member file in individual files
# -- Uses Xarray
def split_ensemble_file(ensemble_file, output_pattern, variable='spell'):
    if not os.path.isdir(os.path.dirname(output_pattern)):
        os.makedirs(os.path.dirname(output_pattern))
    import xarray as xr
    dat = xr.open_dataset(ensemble_file, decode_times=False)[variable]
    i = 0
    for member in dat.time:
        print i
        outfilename = output_pattern+'model'+str(i)+'.nc'
        if not os.path.isfile(outfilename):
            print 'Save '+outfilename
            member_dat = dat[i,:,:]
            wfile = outfilename.replace('.nc','tmp.nc')    
            member_dat.to_netcdf(wfile)
            cmd = 'cdo -b 32 setmissval,1e+20 -copy '+wfile+' '+outfilename+' ; ncrename -v spell,DF6 '+outfilename+' ; rm -f '+wfile
            print cmd
            cmd2 = 'ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" '+outfilename
            print cmd2
            os.system(cmd)
            os.system(cmd2)
        else:
            print outfilename+' already exists'
        i = i + 1

## Start with CMIP6 data

In [21]:
variable='DF6'
CMIP = 'CMIP6'

# -- Compute the annual sums
exp_list = [
    #dict(experiment='ssp585',
    #     clim_period = 'hist'),
    #dict(experiment='ssp585',
    #     clim_period = 'farfut'),
    #dict(experiment='ssp585',
    #     clim_period='midfut'),
    dict(experiment='ssp126',
         clim_period = 'midfut'),
    #dict(experiment='ssp126',
    #     clim_period = 'farfut'),
    #dict(experiment='ssp126',
    #     clim_period = 'hist'),
    #dict(experiment='ssp585',
    #     clim_period = 'farch'),
    #dict(experiment='ssp585',
    #     clim_period='midch'),
    #dict(experiment='ssp126',
    #     clim_period = 'farch'),
    #dict(experiment='ssp126',
    #     clim_period = 'GWL1.5'),
    #dict(experiment='ssp126',
    #     clim_period = 'GWL2.0'),
    #dict(experiment='ssp585',
    #     clim_period = 'GWL1.5'),
    #dict(experiment='ssp585',
    #     clim_period = 'GWL2.0'),
    #dict(experiment='ssp585',
    #     clim_period = 'GWL3.0'),
    #dict(experiment='ssp585',
    #     clim_period = 'GWL4.0'),
]

for exp_dict in exp_list:
    clim_period = exp_dict['clim_period']
    experiment = exp_dict['experiment']
    wfile = '/data/jservon/IPCC/DF/CMIP6/CMIP6_'+experiment+'-DF6-'+clim_period+'.nc'
    output_pattern = '/data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_'+experiment+'_DF6_'+clim_period+'_'
    split_ensemble_file(wfile, output_pattern)

0
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model0.nc
1
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model1.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model1tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model1.nc ; ncrename -v spell,DF

13
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model13.nc
14
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model14.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model14tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model14.nc ; ncrename 

25
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model25.nc
26
Save /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model26.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model26tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_ssp126_DF6_midfut_model26.nc ; ncrename 

## CMIP5

In [22]:
variable='DF6'
CMIP = 'CMIP5'

# -- Compute the annual sums
exp_list = [
    #dict(experiment='rcp85',
    #     clim_period = 'hist'),
    #dict(experiment='rcp85',
    #     clim_period = 'farfut'),
    #dict(experiment='rcp85',
    #     clim_period='midfut'),
    dict(experiment='rcp26',
         clim_period = 'midfut'),
    #dict(experiment='rcp26',
    #     clim_period = 'farfut'),
    #dict(experiment='rcp26',
    #     clim_period = 'hist'),
    #dict(experiment='rcp85',
    #     clim_period = 'farch'),
    #dict(experiment='rcp85',
    #     clim_period='midch'),
    #dict(experiment='rcp26',
    #     clim_period = 'farch'),
    #dict(experiment='rcp26',
    #     clim_period = 'GWL1.5'),
    #dict(experiment='rcp26',
    #     clim_period = 'GWL2.0'),
    #dict(experiment='rcp85',
    #     clim_period = 'GWL1.5'),
    #dict(experiment='rcp85',
    #     clim_period = 'GWL2.0'),
    #dict(experiment='rcp85',
    #     clim_period = 'GWL3.0'),
    #dict(experiment='rcp85',
    #     clim_period = 'GWL4.0'),

]

for exp_dict in exp_list:
    clim_period = exp_dict['clim_period']
    experiment = exp_dict['experiment']
    wfile = '/data/jservon/IPCC/DF/DF-CMIP5/CMIP5_'+experiment+'-DF6-'+clim_period+'.nc'
    output_pattern = '/data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_'+experiment+'_DF6_'+clim_period+'_'
    split_ensemble_file(wfile, output_pattern)

0
Save /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model0.nc
1
Save /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model1.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model1tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model1.nc ; ncrename -v spell,DF6 /data/j

12
Save /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model12.nc
13
Save /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model13.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model13tmp.nc /data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_rcp26_DF6_midfut_model13.nc ; ncrename -v spell,

## CORDEX

In [71]:
variable='DF6'
CMIP = 'CMIP6'

CORDEX_domains = [
    'AFR',
    'AUS',
    'CAM',
    'EAS',
    'NAM',
    'SAM',
    'SEA',
    'WAS'
    # -- Asia
    #'EAS-22','EAS-44',
    #'WAS-22','WAS-44',
    #'SEA-22',
    # -- Europe
    #'EUR-11',
    
]
#AFR  AUS  CAM  EAS  EUR-11  NAM  SAM  SEA  WAS


variable='DF6'
CMIP = 'CORDEX'

# -- Compute the annual sums
exp_list = [
    dict(experiment='rcp85',
         clim_period = 'hist'),
    dict(experiment='rcp85',
         clim_period = 'farfut'),
    dict(experiment='rcp85',
         clim_period='midfut'),
    dict(experiment='rcp26',
         clim_period = 'farfut'),
    dict(experiment='rcp26',
         clim_period = 'hist'),
    dict(experiment='rcp26',
         clim_period = 'midfut'),
    
    dict(experiment='rcp26',
         clim_period = 'GWL1.5'),
    #dict(experiment='rcp26',
    #     clim_period = 'GWL2.0'),
    dict(experiment='rcp85',
         clim_period = 'GWL1.5'),
    dict(experiment='rcp85',
         clim_period = 'GWL2.0'),
    dict(experiment='rcp85',
         clim_period = 'GWL3.0'),
    dict(experiment='rcp85',
         clim_period = 'GWL4.0'),
]

for CORDEX_domain in CORDEX_domains:
    for exp_dict in exp_list:
        clim_period = exp_dict['clim_period']
        experiment = exp_dict['experiment']
        wCORDEX_domain = CORDEX_domain
        if '-' not in CORDEX_domain:
            wCORDEX_domain = CORDEX_domain+'-22'
        wfile = '/data/jservon/IPCC/DF/DF-CORDEX/'+CORDEX_domain+'/'+wCORDEX_domain+'_'+experiment+'-DF6-'+clim_period+'.nc'
        outdir = '/data/jservon/IPCC/DF/individual_models/CORDEX/'+CORDEX_domain
        if not os.path.isdir(outdir):
            os.makedirs(outdir)
        output_pattern = '/data/jservon/IPCC/DF/individual_models/CORDEX/'+wCORDEX_domain+'/'+wCORDEX_domain+'_'+experiment+'_DF6_'+clim_period+'_'
        split_ensemble_file(wfile, output_pattern)
    

0
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model0.nc already exists
1
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model1.nc already exists
2
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model2.nc already exists
3
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model3.nc already exists
4
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model4.nc already exists
5
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model5.nc already exists
6
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model6.nc already exists
7
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model7.nc already exists
8
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_hist_model8.nc already exists
9
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_

2
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model2.nc already exists
3
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model3.nc already exists
4
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model4.nc already exists
5
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model5.nc already exists
6
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model6.nc already exists
7
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model7.nc already exists
8
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model8.nc already exists
9
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model9.nc already exists
10
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_DF6_hist_model10.nc already exists
11
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp26_D

0
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model0.nc already exists
1
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model1.nc already exists
2
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model2.nc already exists
3
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model3.nc already exists
4
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model4.nc already exists
5
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model5.nc already exists
6
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model6.nc already exists
7
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model7.nc already exists
8
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22/AFR-22_rcp85_DF6_GWL3.0_model8.nc already exists
9
/data/jservon/IPCC/DF/individual_models/CORDEX/AFR-22

0
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model0.nc already exists
1
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model1.nc already exists
2
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model2.nc already exists
3
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model3.nc already exists
4
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model4.nc already exists
5
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model5.nc already exists
6
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model6.nc already exists
7
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model7.nc already exists
8
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp26_DF6_farfut_model8.nc already exists
0
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22

0
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model0.nc already exists
1
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model1.nc already exists
2
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model2.nc already exists
3
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model3.nc already exists
4
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model4.nc already exists
5
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model5.nc already exists
6
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model6.nc already exists
7
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model7.nc already exists
8
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22/AUS-22_rcp85_DF6_GWL4.0_model8.nc already exists
9
/data/jservon/IPCC/DF/individual_models/CORDEX/AUS-22

...

14
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14tmp.nc /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model14.nc
15
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model15.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL2.0_model15tmp.nc /data/jservon/IPCC/DF/indivi

0
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0tmp.nc /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model0.nc
1
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model1.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model1tmp.nc /data/jservon/IPCC/DF/individual_model

12
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12tmp.nc /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model12.nc
13
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model13.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model13tmp.nc /data/jservon/IPCC/DF/indivi

24
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24tmp.nc /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model24.nc
25
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model25.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL3.0_model25tmp.nc /data/jservon/IPCC/DF/indivi

10
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10tmp.nc /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10.nc ; rm -f /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10tmp.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model10.nc
11
Save /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model11.nc
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/individual_models/CORDEX/WAS-22/WAS-22_rcp85_DF6_GWL4.0_model11tmp.nc /data/jservon/IPCC/DF/indivi

In [77]:
CORDEX_domain = 'EUR-11'

exp_list = [
    #dict(experiment='rcp85',
    #     clim_period = 'ref'),
    #dict(experiment='rcp85',
    #     clim_period = 'far'),
    #dict(experiment='rcp85',
    #     clim_period='mid'),
    #dict(experiment='rcp26',
    #     clim_period = 'far'),
    #dict(experiment='rcp26',
    #     clim_period = 'ref'),
    dict(experiment='rcp26',
         clim_period = 'mid'),
    
    #dict(experiment='GWLs',
    #     clim_period = '1.5deg'),
    #dict(experiment='GWLs',
    #     clim_period = '2.0deg'),
    #dict(experiment='GWLs',
    #     clim_period = '4.0deg'),
]


#exp_list = [
#    dict(experiment='rcp85',
#         clim_period = 'far'),
#    dict(experiment='GWLs',
#         clim_period = '1.5deg'),
#]

def prepare_EUR_files(wfile, outfilename, variable='spell'):
    if not os.path.isfile(outfilename):
        cmd = 'cdo -b 32 setmissval,1e+20 -copy '+wfile+' '+outfilename+' ; ncrename -v spell,DF6 '+outfilename
        print cmd
        cmd2 = 'ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" '+outfilename
        print cmd2
        os.system(cmd)
        os.system(cmd2)
    else:
        print outfilename+' already exists'
#
for exp in exp_list:
    experiment  = exp['experiment']
    clim_period = exp['clim_period']
    
    if experiment=='GWLs':
        file_pattern = '/data/jservon/IPCC/DF/DF-CORDEX/'+CORDEX_domain+'/'+experiment+'/pr_mon_*_OK_SPI6_sel*_SPELL_'+clim_period+'.nc'
        wclim_period = 'GWL'+clim_period.replace('deg','')
        output_pattern = '/data/jservon/IPCC/DF/individual_models/CORDEX/'+CORDEX_domain+'/'+CORDEX_domain+'_rcptarget_DF6_'+wclim_period+'_'
    else:
        if clim_period not in ['ref']:
            wclim_period = clim_period+'fut'
        else:
            wclim_period = 'hist'
        file_pattern = '/data/jservon/IPCC/DF/DF-CORDEX/'+CORDEX_domain+'/'+experiment+'/pr_mon_*_OK_SPI6_sel_SPELL_'+clim_period+'.nc'
        output_pattern = '/data/jservon/IPCC/DF/individual_models/CORDEX/'+CORDEX_domain+'/'+CORDEX_domain+'_'+experiment+'_DF6_'+wclim_period+'_'
    
    lof = glob.glob(file_pattern)
    i = 0
    for wfile in lof:
        wexperiment = os.path.basename(wfile).split('_')[5]
        outfilename = output_pattern.replace('rcptarget',wexperiment)+'model'+str(i)+'.nc'
        if not os.path.isdir(os.path.dirname(outfilename)):
            os.makedirs(os.path.dirname(outfilename))
            print 'creating '+os.path.dirname(outfilename)
        print '--> '+experiment+' '+clim_period
        prepare_EUR_files(wfile, outfilename)
        i = i + 1         


--> rcp26 mid
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/DF-CORDEX/EUR-11/rcp26/pr_mon_MPI-M-MPI-ESM-LR_r1_KNMI-RACMO22E_full26_OK_SPI6_sel_SPELL_mid.nc /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model0.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model0.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model0.nc
--> rcp26 mid
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/DF-CORDEX/EUR-11/rcp26/pr_mon_CNRM-CERFACS-CNRM-CM5_r1_RMIB-UGent-ALARO-0_full26_OK_SPI6_sel_SPELL_mid.nc /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model1.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model1.nc
ncatted -O -a coordinates,DF6,o,c,"

--> rcp26 mid
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/DF-CORDEX/EUR-11/rcp26/pr_mon_NOAA-GFDL-GFDL-ESM2G_r1_GERICS-REMO2015_full26_OK_SPI6_sel_SPELL_mid.nc /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model14.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model14.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model14.nc
--> rcp26 mid
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/DF-CORDEX/EUR-11/rcp26/pr_mon_CNRM-CERFACS-CNRM-CM5_r1_GERICS-REMO2015_full26_OK_SPI6_sel_SPELL_mid.nc /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model15.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model15.nc
ncatted -O -a coordinates,D

--> rcp26 mid
cdo -b 32 setmissval,1e+20 -copy /data/jservon/IPCC/DF/DF-CORDEX/EUR-11/rcp26/pr_mon_MPI-M-MPI-ESM-LR_r2_MPI-CSC-REMO2009_full26_OK_SPI6_sel_SPELL_mid.nc /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model28.nc ; ncrename -v spell,DF6 /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model28.nc
ncatted -O -a coordinates,DF6,o,c,"lat lon" -a units,DF6,o,c,"Droughts" -a long_name,DF6,o,c,"Number of droughts per decade (DF6 index)" /data/jservon/IPCC/DF/individual_models/CORDEX/EUR-11/EUR-11_rcp26_DF6_midfut_model28.nc


In [61]:
lof

['/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_MPI-M-MPI-ESM-LR_r2_MPI-CSC-REMO2009_rcp85_OK_SPI6_sel53_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_ICHEC-EC-EARTH_r12_GERICS-REMO2015_rcp26_OK_SPI6_sel07_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_MPI-M-MPI-ESM-LR_r1_CNRM-ALADIN63_rcp85_OK_SPI6_sel44_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_NCC-NorESM1-M_r1_KNMI-RACMO22E_rcp85_OK_SPI6_sel63_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_CNRM-CERFACS-CNRM-CM5_r1_GERICS-REMO2015_rcp85_OK_SPI6_sel07_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_NCC-NorESM1-M_r1_SMHI-RCA4_rcp85_OK_SPI6_sel65_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_MPI-M-MPI-ESM-LR_r3_GERICS-REMO2015_rcp85_OK_SPI6_sel56_SPELL_1.5deg.nc',
 '/data/jservon/IPCC/DF/DF-CORDEX/EUR-11/GWLs/pr_mon_MPI-M-MPI-ESM-LR_r1_KNMI-RACMO22E_rcp26_OK_SPI6_sel22_SPELL_1.5deg.nc

### Declare CliMAF project to get the data

In [5]:
pattern = '/data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_${experiment}_DF6_${clim_period}_${member}.nc'
#pattern = '/data/jservon/IPCC/DF/CMIP6/CMIP6_${experiment}_DF6_${clim_period}_${member}.nc'
cproject('DF_cmip6_ch12','experiment','clim_period',('period','fx'),'member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cmip6_ch12', url=pattern)

pattern = '/data/jservon/IPCC/DF/individual_models/CMIP6/CMIP6_${experiment}_DF6_${GWL}_${member}.nc'
cproject('DF_cmip6_GWL_ch12','experiment','GWL',('period','fx'),'member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cmip6_GWL_ch12', url=pattern)

<climaf.dataloc.dataloc instance at 0x2b909cf17200>

In [6]:
pattern = '/data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_${experiment}_DF6_${clim_period}_${member}.nc'
cproject('DF_cmip5_ch12','experiment','clim_period',('period','fx'),'member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cmip5_ch12', url=pattern)

pattern = '/data/jservon/IPCC/DF/individual_models/CMIP5/CMIP5_${experiment}_DF6_${GWL}_${member}.nc'
cproject('DF_cmip5_GWL_ch12','experiment','GWL',('period','fx'),'member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cmip5_GWL_ch12', url=pattern)

<climaf.dataloc.dataloc instance at 0x2b909d438758>

In [3]:
pattern = '/data/jservon/IPCC/DF/individual_models/CORDEX/${CORDEX_domain}/${CORDEX_domain}_${experiment}_DF6_${clim_period}_${member}.nc'
pattern_EUR = '/data/jservon/IPCC/DF/CORDEX/${CORDEX_domain}/${experiment}/pr_mon_${member}_OK_SPI6_sel_SPELL_${clim_period}.nc'
cproject('DF_cordex_ch12','experiment','clim_period',('period','fx'),'CORDEX_domain','member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cordex_ch12', url=[pattern,pattern_EUR])

pattern_GWL = '/data/jservon/IPCC/DF/individual_models/CORDEX/${CORDEX_domain}/${CORDEX_domain}_${experiment}_DF6_${clim_period}_${member}.nc'
pattern_EUR_GWL = '/data/jservon/IPCC/DF/CORDEX/${CORDEX_domain}/GWLs/pr_mon_${member}_OK_SPI6_sel_SPELL_${GWL}deg.nc'
cproject('DF_cordex_GWL_ch12','experiment','clim_period',('period','fx'),'CORDEX_domain','member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cordex_GWL_ch12', url=[pattern_GWL,pattern_EUR_GWL])

<climaf.dataloc.dataloc instance at 0x2b4abc6b3518>

In [4]:
def weighted_mean(da, weights, dim):
    """Reduce da by a weighted mean along some dimension(s).

    Parameters
    ----------
    da : DataArray
        Object over which the weighted reduction operation is applied.    
    weights : DataArray
        An array of weights associated with the values in this Dataset.
    dim : str or sequence of str, optional
        Dimension(s) over which to apply the weighted `mean`.
        
    Returns
    -------
    weighted_mean : DataArray
        New DataArray with weighted mean applied to its data and
        the indicated dimension(s) removed.
    """

    weighted_sum = (da * weights).sum(dim=dim, skipna=True)
    # need to mask weights where data is not valid
    masked_weights = weights.where(da.notnull())
    sum_of_weights = masked_weights.sum(dim=dim, skipna=True)
    valid_weights = sum_of_weights != 0
    sum_of_weights = sum_of_weights.where(valid_weights)

    return weighted_sum / sum_of_weights

def average_over_AR6_region(filename, variable, region_name):

    # -- AR6 regions
    #ar6_all = regionmask.defined_regions.ar6.all
    # -- Get the regions
    ar6_land = regionmask.defined_regions.ar6.land

    #ax = ar6_all.plot()
    # -- Get land/sea mask (generic)
    land_110 = regionmask.defined_regions.natural_earth.land_110

    # -- Get data
    ds = xr.open_dataset(filename, decode_times=False)
    dat = ds[variable]
    dat.values = np.array(dat.values, dtype=np.float32)

    # -- Mask the data
    mask_3D = ar6_land.mask_3D(dat) # AR6 mask
    land_mask = land_110.mask_3D(dat) # Land sea mask
    mask_lsm = mask_3D * land_mask.squeeze(drop=True) # Combine the two

    weights = np.cos(np.deg2rad(dat.lat))
    
    if region_name=='all':
        tmp = weighted_mean(dat, mask_lsm * weights, ("lon", "lat"))
        tmp['abbrevs'] = mask_3D.abbrevs
        return tmp
    else:
        # -- Case dat is has time dim
        if 'time' in dat.dims:
            matlat = np.mean(dat.values, axis=dat.dims.index('time')) * 0
        else:
            matlat = dat.values * 0

        if dat.dims.index('lat')<dat.dims.index('lon'):
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[:,i] = dat.lat
        else:
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[i,:] = dat.lat
    
        weights = np.cos(np.deg2rad(matlat))

        if isinstance(region_name, list):
            res = list()
            for region in region_name:
                region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region))
                dat_region = dat.where(region_mask)
                weights_region = np.where(region_mask, weights, float("nan"))
                #weights_region = weights.where(region_mask)
                res.append( weighted_mean(dat_region, region_mask*weights_region, ("lon","lat")) )
            return res
        else:
            region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region_name))
            dat_region = dat.where(region_mask)
            weights_region = np.where(region_mask, weights, float("nan"))
            #weights_region = weights.where(region_mask)            
            return weighted_mean(dat_region, region_mask*weights_region, ("lon","lat"))
#    
#
def average_over_AR6_region_new(filename, variable, region_name):

    # -- AR6 regions
    #ar6_all = regionmask.defined_regions.ar6.all
    # -- Get the regions
    ar6_land = regionmask.defined_regions.ar6.land

    #ax = ar6_all.plot()
    # -- Get land/sea mask (generic)
    land_110 = regionmask.defined_regions.natural_earth.land_110

    # -- Get data
    ds = xr.open_dataset(filename, decode_times=False)
    dat = ds[variable]
    dat.values = np.array(dat.values, dtype=np.float32)

    # -- Mask the data
    mask_3D = ar6_land.mask_3D(dat) # AR6 mask
    land_mask = land_110.mask_3D(dat) # Land sea mask
    mask_lsm = mask_3D * land_mask.squeeze(drop=True) # Combine the two
    
    # -- Compute weights
    if dat.lat.shape == dat.shape:
        weights = np.cos(np.deg2rad(dat.lat))
    else:
        # -- Case dat is has time dim
        if 'time' in dat.dims:
            matlat = np.mean(dat.values, axis=dat.dims.index('time')) * 0
        else:
            matlat = dat.values * 0

        if dat.dims.index('lat')<dat.dims.index('lon'):
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[:,i] = dat.lat
        else:
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[i,:] = dat.lat
    
        weights = np.cos(np.deg2rad(matlat))
    
    if region_name=='all':
        return weighted_mean(dat, mask_lsm * weights, ("lon", "lat"))
    else:
        if isinstance(region_name, list):
            res = list()
            for region in region_name:
                region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region))
                dat_region = dat.where(region_mask)
                weights_region = np.where(region_mask, weights, float("nan"))
                #weights_region = weights.where(region_mask)
                res.append( weighted_mean(dat_region, region_mask*weights_region, ("lon","lat")) )
            return res
        else:
            region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region_name))
            dat_region = dat.where(region_mask)
            weights_region = np.where(region_mask, weights, float("nan"))
            #weights_region = weights.where(region_mask)            
            return weighted_mean(dat_region, region_mask*weights_region, ("lon","lat"))
#
def regions_contained(lon, lat, regions):

    # determine if the longitude needs to be wrapped
    regions_is_180 = regions.lon_180
    grid_is_180 = regionmask.core.utils._is_180(lon.min(), lon.max())

    wrap_lon = not regions_is_180 == grid_is_180

    lon_orig = lon.copy()
    if wrap_lon:
        lon = regionmask.core.utils._wrapAngle(lon, wrap_lon)

    lon = np.asarray(lon).squeeze()
    lat = np.asarray(lat).squeeze()

    if lon.ndim == 1 and lat.ndim == 1:
        poly = shapely.geometry.box(lon.min(), lat.min(), lon.max(), lat.max())

    # convex_hull is not really what we need
    # https://gist.github.com/dwyerk/10561690
    #     elif lon.ndim == 2 and lat.ndim == 2:
    #         # get the convex hull from all points
    #         lonlat = np.stack([lon.ravel(), lat.ravel()], axis=1)
    #         multipoint = shapely.geometry.MultiPoint(lonlat)
    #         poly = multipoint.convex_hull
    else:
        raise ValueError("Cannot currently handle 2D coordinates")

    fully_contained = list()
    for region_poly in regions.polygons:
        res = poly.contains(region_poly)

        fully_contained.append(res)

    return xr.DataArray(
        fully_contained, dims=["region"], coords=dict(region=regions.numbers)
    )

if None:
    region_name = "all"
    variable = 'tx35'
    filename = "/data/jservon/IPCC/tx35/individual_models/CMIP6_ssp585_tx35_2100_NorESM2-LM_r1i1p1f1.nc"

    tmp = average_over_AR6_region(filename, variable, region_name)
    tmp

In [18]:
!ls /data/jservon/IPCC/DF/individual_models/CMIP6

CMIP6_ssp126_DF6_farch_model0.nc    CMIP6_ssp585_DF6_GWL1.5_model18.nc
CMIP6_ssp126_DF6_farch_model10.nc   CMIP6_ssp585_DF6_GWL1.5_model19.nc
CMIP6_ssp126_DF6_farch_model11.nc   CMIP6_ssp585_DF6_GWL1.5_model1.nc
CMIP6_ssp126_DF6_farch_model12.nc   CMIP6_ssp585_DF6_GWL1.5_model20.nc
CMIP6_ssp126_DF6_farch_model13.nc   CMIP6_ssp585_DF6_GWL1.5_model21.nc
CMIP6_ssp126_DF6_farch_model14.nc   CMIP6_ssp585_DF6_GWL1.5_model22.nc
CMIP6_ssp126_DF6_farch_model15.nc   CMIP6_ssp585_DF6_GWL1.5_model23.nc
CMIP6_ssp126_DF6_farch_model16.nc   CMIP6_ssp585_DF6_GWL1.5_model24.nc
CMIP6_ssp126_DF6_farch_model17.nc   CMIP6_ssp585_DF6_GWL1.5_model25.nc
CMIP6_ssp126_DF6_farch_model18.nc   CMIP6_ssp585_DF6_GWL1.5_model26.nc
CMIP6_ssp126_DF6_farch_model19.nc   CMIP6_ssp585_DF6_GWL1.5_model27.nc
CMIP6_ssp126_DF6_farch_model1.nc    CMIP6_ssp585_DF6_GWL1.5_model28.nc
CMIP6_ssp126_DF6_farch_model20.nc   CMIP6_ssp585_DF6_GWL1.5_model29.nc
CMIP6_ssp126_DF6_farch_model21.nc   CMIP6_ssp585_DF6_GWL1.5_model2.nc
CMIP6_ss

### Start with baseline, mid-term, late-term, ssp126 and ssp585

In [7]:
exp_list = [
    # -- Baseline (ssp126 and ssp585 are the same files)
    dict(experiment='ssp585',
         clim_period = 'hist'),    
    # -- Mid term
    dict(experiment='ssp585',
         clim_period = 'midfut'),
    dict(experiment='ssp126',
         clim_period = 'midfut'),
    # -- Late term
    dict(experiment='ssp585',
         clim_period = 'farfut'),
    dict(experiment='ssp126',
         clim_period = 'farfut'),
]

# -- Loop on the scenarios
ens_exp_dict = dict()
clog('critical')
for exp in exp_list:
    #
    # -- Experiment and period
    experiment = exp['experiment']
    clim_period = exp['clim_period']
    
    # -- Create ensemble object for the scenario
    req_exp = ds(project='DF_cmip6_ch12',
                 experiment = experiment,
                 clim_period = clim_period,
                 member = '*'
                )
    ens_exp = req_exp.explore('ensemble')
    
    # -- Climatologies
    ens_exp_dict[experiment+'_'+clim_period] = clim_average(ens_exp, 'ANM')
    

### Then GWLs (mix ssp126 and ssp585)

In [8]:
# -- Loop on the scenarios
ens_GWL_dict = dict()
for GWL in ['GWL1.5','GWL2.0','GWL3.0','GWL4.0']:
    #
    if GWL in ['GWL1.5','GWL2.0']:
        # -- Create ensemble object for the scenario
        req_ssp126 = ds(project='DF_cmip6_GWL_ch12',
                     experiment = 'ssp126',
                     GWL = GWL,
                     member = '*'
                    )
        ens_ssp126 = req_ssp126.explore('ensemble')

        # -- Create ensemble object for the scenario
        req_ssp585 = ds(project='DF_cmip6_GWL_ch12',
                     experiment = 'ssp585',
                     GWL = GWL,
                     member = '*'
                    )
        ens_ssp585 = req_ssp585.explore('ensemble')
        #
        # -- Merge ensembles
        GWL_ens = merge_climaf_ensembles([add_prefix_suffix_to_ens_req(ens_ssp126,suffix='_ssp126'),
                                          add_prefix_suffix_to_ens_req(ens_ssp585,suffix='_ssp585')])
    else:
        # -- Create ensemble object for the scenario
        req_ssp585 = ds(project='DF_cmip6_GWL_ch12',
                     experiment = 'ssp585',
                     GWL = GWL,
                     member = '*'
                    )
        GWL_ens = req_ssp585.explore('ensemble')
        
    # -- Climatologies
    ens_GWL_dict[GWL] = clim_average(GWL_ens, 'ANM')
    

In [9]:
regional_averages = dict()

# -- Loop on experiments / horizons
for ens_exp in ens_exp_dict:
    print ens_exp
    regional_averages[ens_exp] = dict()
    # -- Loop on the members of each ensemble
    for mem in ens_exp_dict[ens_exp]:
        print mem
        # -- Compute the averages for each AR6 region thanks to regionmask
        #tmp = average_over_AR6_region(cfile(ens_exp_dict[ens_exp][mem]), 'DF6', 'all')
        #region_names = tmp.abbrevs
        #for tmp_region_name in region_names:
        #    region_name = str(tmp_region_name.values)
        #    print region_name
        #    region_value = float(tmp.sel(region=list(tmp.abbrevs).index(region_name)).values)
        #    if region_name not in regional_averages[ens_exp]:
        #        regional_averages[ens_exp][region_name] = [region_value]
        #    else:
        #        regional_averages[ens_exp][region_name].append(region_value)
        tmp = average_over_AR6_region(cfile(ens_exp_dict[ens_exp][mem]), 'DF6', 'all')
        region_names = list(tmp.abbrevs)
        ttmp = average_over_AR6_region(cfile(ens_exp_dict[ens_exp][mem]), 'DF6', region_names)
        for tmp_region_name in region_names:
            region_name = str(tmp_region_name.values)
            region_value = float(ttmp[list(tmp.abbrevs).index(region_name)])
            print region_name, region_value
            if region_name not in regional_averages[ens_exp]:
                regional_averages[ens_exp][region_name] = [region_value]
            else:
                regional_averages[ens_exp][region_name].append(region_value)


ssp126_midfut
model9
GIC 1.94517822596
NWN 2.55808782859
NEN 2.1792018266
WNA 3.52818276022
CNA 4.37345787507
ENA 3.78129311677
NCA 4.33097645941
SCA 3.05995147995
CAR 3.3788285374
NWS 2.40373560337
NSA 3.92183559821
NES 4.67375588079
SAM 2.94729704066
SWS 4.68876813874
SES 3.43994742805
SSA 5.67514866205
NEU 3.19891701597
WCE 3.7062897818
EEU 3.16210330977
MED 4.06397191392
SAH 3.72529170461
WAF 1.55038306717
CAF 1.7710417489
NEAF 0.895130303457
SEAF 2.07083210316
WSAF 4.61263494211
ESAF 3.44265140599
MDG 3.47770303235
RAR 1.65658110624
WSB 2.86729671118
ESB 2.27985207419
RFE 2.08446490688
WCA 3.74189913206
ECA 3.49969604247
TIB 2.52685261737
EAS 2.53720907433
ARP 2.76560447887
SAS 2.06784277681
SEA 2.67221297537
NAU 4.05176559079
CAU 5.26531936823
EAU 5.59551323222
SAU 5.18408032938
NZ 3.38523911474
EAN 2.44039667379
WAN 2.60460460167
model8
GIC 3.24496339251
NWN 2.11905365115
NEN 2.73133603592
WNA 3.34816571628
CNA 4.08680302525
ENA 3.53512504986
NCA 3.8264220324
SCA 2.79531532621
C

SEA 4.5072408439
NAU 5.41746210187
CAU 6.23017021804
EAU 5.44204667095
SAU 6.21560422369
NZ 2.54137519528
EAN 3.16217707945
WAN 4.13456071717
model13
GIC 2.06994069485
NWN 1.94458673159
NEN 1.90640958711
WNA 3.05099998335
CNA 3.76214501476
ENA 3.1008565227
NCA 5.17434926487
SCA 2.09314453728
CAR 3.84392690661
NWS 2.79257752946
NSA 2.9816714439
NES 3.18669297111
SAM 3.60401749192
SWS 5.49369587901
SES 2.95437402642
SSA 4.55918752605
NEU 2.60561986213
WCE 3.45757185738
EEU 2.46476455748
MED 4.55265990329
SAH 2.912189451
WAF 2.06168229077
CAF 2.36405604837
NEAF 2.51634023579
SEAF 3.92459626994
WSAF 3.92151676456
ESAF 4.24691974145
MDG 4.86212886196
RAR 1.31021232196
WSB 1.83706127693
ESB 1.09384420999
RFE 1.21911462692
WCA 3.46670626808
ECA 2.05520333792
TIB 1.62647544218
EAS 3.29824310661
ARP 2.74516510368
SAS 2.39601823228
SEA 3.21323384946
NAU 4.37482885515
CAU 4.92144354506
EAU 3.680961909
SAU 4.63744705706
NZ 3.21295871699
EAN 2.95724326999
WAN 3.56483863527
model12
GIC 3.18685769793

ECA 1.55549918451
TIB 1.75525906652
EAS 2.39704458852
ARP 2.00708625397
SAS 1.93201211742
SEA 3.67783498929
NAU 5.6133942133
CAU 4.91718358535
EAU 5.61856915831
SAU 4.82737204627
NZ 2.37406776731
EAN 2.04729338586
WAN 2.51601267813
model19
GIC 1.26619447388
NWN 1.40760120936
NEN 1.3857214088
WNA 4.34033561141
CNA 3.98500269096
ENA 2.87745476885
NCA 4.00495674343
SCA 3.40996605239
CAR 3.13819056477
NWS 4.31191712687
NSA 2.34024045964
NES 2.23180949069
SAM 3.29517480703
SWS 6.03567280215
SES 6.28181914051
SSA 4.73703363878
NEU 2.31758710488
WCE 3.68656720578
EEU 1.87804168313
MED 4.96148843175
SAH 1.54889752995
WAF 1.35712639274
CAF 3.40425154808
NEAF 2.42154360632
SEAF 3.59104104592
WSAF 5.75619790801
ESAF 4.81434987031
MDG 5.19250564564
RAR 0.879132959787
WSB 2.48092308449
ESB 2.00922343765
RFE 1.68397110914
WCA 3.17582813008
ECA 2.7069980527
TIB 2.21811365231
EAS 2.69854850179
ARP 1.36775187598
SAS 3.19326228012
SEA 3.67028826025
NAU 2.72494392201
CAU 4.69156334287
EAU 4.49699166708
S

MDG 4.23973670117
RAR 1.61520067684
WSB 3.38982890117
ESB 1.96395986292
RFE 2.01352848091
WCA 4.13883253093
ECA 1.92156287027
TIB 4.00584535498
EAS 2.52280364702
ARP 5.02670061517
SAS 4.63284924642
SEA 4.43776760444
NAU 5.81326788136
CAU 6.37248272858
EAU 6.73262355285
SAU 5.96056098323
NZ 3.78601555157
EAN 1.78352551502
WAN 2.43616929099
model29
GIC 2.11189672441
NWN 1.71146880039
NEN 1.43494848853
WNA 1.83687742565
CNA 2.54573459918
ENA 2.31336794571
NCA 2.79040843954
SCA 3.80146970744
CAR 5.28499484834
NWS 4.3974119455
NSA 5.28165405809
NES 6.6925717518
SAM 5.14002858442
SWS 3.72942976304
SES 3.55679647533
SSA 3.10989467494
NEU 3.34246150791
WCE 3.76110166917
EEU 3.14301666998
MED 4.14326871126
SAH 2.90706467305
WAF 0.989771273808
CAF 3.13607445106
NEAF 1.28768255184
SEAF 2.65713797625
WSAF 5.95597816499
ESAF 6.01589249454
MDG 5.42025373184
RAR 1.50288937482
WSB 3.78351200915
ESB 1.99775974918
RFE 1.49336662344
WCA 3.08850173008
ECA 2.30987824801
TIB 1.84063662736
EAS 3.79217477274


MED 5.92729375758
SAH 1.90704675063
WAF 2.37099122248
CAF 1.80100378173
NEAF 1.30246086643
SEAF 3.31841606475
WSAF 5.32874421549
ESAF 4.86729788793
MDG 4.85054740031
RAR 1.05001704913
WSB 2.62934819306
ESB 2.63646118178
RFE 1.79215702446
WCA 2.6772339336
ECA 0.902465253065
TIB 3.34383786757
EAS 4.70848196568
ARP 3.96162195748
SAS 8.40660821845
SEA 5.16999391114
NAU 9.41926404948
CAU 9.66658413759
EAU 9.443590567
SAU 7.28829069577
NZ 2.52083489884
EAN 3.13138042201
WAN 5.10368034903
model4
GIC 3.63366081221
NWN 1.70711993114
NEN 1.70875130901
WNA 3.09773023215
CNA 2.57267541158
ENA 2.4708735866
NCA 3.71301734018
SCA 6.69008630389
CAR 4.98204680328
NWS 2.23213631392
NSA 6.30047861606
NES 7.63993139903
SAM 4.88443141303
SWS 5.19501883944
SES 3.39255637238
SSA 5.53092306977
NEU 1.25498513862
WCE 2.64749407053
EEU 2.49085116665
MED 5.33586223981
SAH 4.19905397199
WAF 7.00941686976
CAF 3.46810294943
NEAF 3.38716470731
SEAF 4.00330939984
WSAF 6.16524562594
ESAF 6.44736985467
MDG 5.82987920483

...

NEAF 4.15720433332
SEAF 3.6872023121
WSAF 3.90880280509
ESAF 3.50288133723
MDG 3.64881135707
RAR 2.98762869767
WSB 2.88048215553
ESB 3.27517638449
RFE 2.92595769118
WCA 3.18858444774
ECA 3.50418004127
TIB 2.57819102518
EAS 4.05048971897
ARP 3.57975316009
SAS 3.29523517159
SEA 3.53521320213
NAU 3.60194865567
CAU 4.21834169935
EAU 4.91611415773
SAU 4.53358772786
NZ 3.36764316052
EAN 4.08181856013
WAN 3.25773733914
model28
GIC 2.94549580444
NWN 2.66346765277
NEN 2.67011188831
WNA 4.09242961896
CNA 2.83612338159
ENA 3.23194180322
NCA 2.78069624826
SCA 4.03661207508
CAR 4.61865563311
NWS 2.96285464696
NSA 3.7595455044
NES 4.53541650524
SAM 4.25146276331
SWS 3.81080994644
SES 4.1190075888
SSA 3.87591218265
NEU 3.99707901324
WCE 4.01593484968
EEU 3.45579205259
MED 4.38191886214
SAH 3.2184653308
WAF 2.61387647518
CAF 3.45081852832
NEAF 3.34825864854
SEAF 2.93199151579
WSAF 3.43847233791
ESAF 3.50470892997
MDG 4.09767293289
RAR 2.89713945325
WSB 3.16631592232
ESB 2.74933041139
RFE 2.80677171046

NEU 2.95796034369
WCE 3.23901482995
EEU 3.4494249061
MED 4.03918029735
SAH 3.5026911202
WAF 2.06310725182
CAF 2.28279080235
NEAF 3.31968049225
SEAF 3.39099907131
WSAF 3.7236932498
ESAF 3.97795927375
MDG 3.4780408965
RAR 2.656231811
WSB 3.23530875544
ESB 2.84998060152
RFE 2.73159682144
WCA 3.59812161753
ECA 2.77885774963
TIB 2.54044545812
EAS 3.91486171876
ARP 2.56600269807
SAS 3.14048680431
SEA 3.27145945607
NAU 4.47103394836
CAU 5.03441128814
EAU 3.7400041994
SAU 4.11461911795
NZ 3.62120921248
EAN 3.4180303118
WAN 2.93406546452
ssp126_farfut
model9
GIC 1.59285185715
NWN 2.09147501945
NEN 2.22849223043
WNA 3.41144139813
CNA 3.31314224428
ENA 3.01913810022
NCA 4.02057793995
SCA 2.22652205438
CAR 2.76482412957
NWS 2.82384747185
NSA 4.26836755832
NES 5.92260208034
SAM 3.95021217122
SWS 4.74229948326
SES 3.21739606558
SSA 4.15368429667
NEU 3.31489619584
WCE 3.23729309562
EEU 2.36291871672
MED 3.72382638405
SAH 3.43092079187
WAF 1.36197644115
CAF 1.88416320216
NEAF 1.59395751413
SEAF 3.3151

CAR 3.36489608459
NWS 3.10060834837
NSA 6.95995171104
NES 6.97637022558
SAM 5.47655246486
SWS 3.21607451568
SES 3.6860017891
SSA 2.67759864125
NEU 1.87706010503
WCE 2.3622745756
EEU 2.93803385438
MED 3.55456790526
SAH 3.71106647356
WAF 4.31356799725
CAF 2.35524887059
NEAF 2.56867808028
SEAF 2.44879205318
WSAF 4.98986370658
ESAF 5.35664164025
MDG 4.57627926712
RAR 1.45593551687
WSB 2.7601978191
ESB 2.26033514056
RFE 2.59480468233
WCA 3.33875275986
ECA 2.23785863035
TIB 1.97117369883
EAS 2.25783449956
ARP 3.43669894483
SAS 3.96699755398
SEA 3.16085340693
NAU 5.27507610481
CAU 5.70247479849
EAU 5.92355408988
SAU 5.66583083486
NZ 3.19330899865
EAN 2.67792631259
WAN 4.10737013136
model13
GIC 2.05039768811
NWN 1.55632291808
NEN 1.46136375364
WNA 3.78567406112
CNA 2.75980447249
ENA 2.5192856633
NCA 4.26774327139
SCA 2.75141199612
CAR 2.73751779544
NWS 2.8718319554
NSA 3.35338107652
NES 3.63344467099
SAM 3.74650630826
SWS 5.51645573268
SES 2.92071639463
SSA 4.96514616768
NEU 2.13478225917
WCE 

GIC 0.832796015398
NWN 1.3499796706
NEN 1.30612602863
WNA 3.08181512552
CNA 4.98683072185
ENA 2.96421047838
NCA 3.34080807263
SCA 4.53271558153
CAR 4.33887588729
NWS 4.96328715832
NSA 6.09925769631
NES 6.93259997246
SAM 6.80425941357
SWS 4.70197981908
SES 3.73327345291
SSA 3.49981887538
NEU 2.54424294607
WCE 2.9223474851
EEU 1.58069618876
MED 4.63259653571
SAH 3.43131749773
WAF 1.68714562948
CAF 2.95342210164
NEAF 1.52815604433
SEAF 2.97438082551
WSAF 5.78262749871
ESAF 5.75572455021
MDG 5.51564330064
RAR 0.499324767925
WSB 1.03355398795
ESB 0.706878346773
RFE 0.668106238994
WCA 1.76440418628
ECA 1.22375613643
TIB 1.10457113772
EAS 2.00577097074
ARP 1.50756587656
SAS 1.37827358415
SEA 3.81793254809
NAU 5.44173460162
CAU 4.54614593218
EAU 5.62983095367
SAU 4.04947751288
NZ 2.64239748372
EAN 1.75799368268
WAN 2.80662226582
model19
GIC 2.11127651043
NWN 1.55633340436
NEN 1.50845213927
WNA 3.28008453657
CNA 3.08807353434
ENA 2.62300740574
NCA 3.96885944531
SCA 3.10186784649
CAR 3.642488731

NAU 4.86322372339
CAU 6.1717667376
EAU 5.71702773249
SAU 6.5594187622
NZ 2.39658995852
EAN 1.61421453907
WAN 1.36355459065
model27
GIC 4.17567448471
NWN 1.49554137438
NEN 1.51070934223
WNA 2.23896667682
CNA 2.60912308916
ENA 2.893405594
NCA 4.59019278402
SCA 6.67700581815
CAR 4.89926849486
NWS 3.45399441332
NSA 6.10554885352
NES 5.16904276047
SAM 5.43114703281
SWS 4.70124283
SES 2.84348824657
SSA 5.01220805358
NEU 4.20877852918
WCE 3.91729476785
EEU 4.56969708846
MED 5.28548700244
SAH 4.59831791201
WAF 5.87567100226
CAF 3.83688997093
NEAF 3.18104314524
SEAF 2.42894256587
WSAF 5.74034732302
ESAF 5.34480813537
MDG 2.84057738853
RAR 1.9300405372
WSB 3.49296841661
ESB 1.35917208715
RFE 1.64267851537
WCA 3.70036203399
ECA 1.91961678505
TIB 3.68006880563
EAS 2.62922155236
ARP 3.86101115066
SAS 4.40508861842
SEA 4.3809338924
NAU 5.94908415163
CAU 5.99054831865
EAU 6.09476228118
SAU 6.04741629448
NZ 2.51633297425
EAN 0.724910185795
WAN 1.24894964221
model29
GIC 0.893200438295
NWN 0.69300442213

In [11]:
for GWL in ens_GWL_dict :
    print GWL
    regional_averages[GWL] = dict()
    # -- Loop on the members of each ensemble
    for mem in ens_GWL_dict[GWL]:
        print mem
        # -- Compute the averages for each AR6 region thanks to regionmask
        #tmp = average_over_AR6_region(cfile(ens_GWL_dict[GWL][mem]), 'DF6', 'all')
        #region_names = tmp.abbrevs
        #for tmp_region_name in region_names:
        #    region_name = str(tmp_region_name.values)
        #    print region_name
        #    region_value = float(tmp.sel(region=list(tmp.abbrevs).index(region_name)).values)
        #    if region_name not in regional_averages[GWL]:
        #        regional_averages[GWL][region_name] = [region_value]
        #    else:
        #        regional_averages[GWL][region_name].append(region_value)
        tmp = average_over_AR6_region(cfile(ens_GWL_dict[GWL][mem]), 'DF6', 'all')
        region_names = list(tmp.abbrevs)
        ttmp = average_over_AR6_region(cfile(ens_GWL_dict[GWL][mem]), 'DF6', region_names)
        for tmp_region_name in region_names:
            region_name = str(tmp_region_name.values)
            region_value = float(ttmp[list(tmp.abbrevs).index(region_name)])
            print region_name, region_value
            if region_name not in regional_averages[GWL]:
                regional_averages[GWL][region_name] = [region_value]
            else:
                regional_averages[GWL][region_name].append(region_value)


GWL2.0
model4_ssp126
GIC 1.70750853377
NWN 1.60204588683
NEN 1.37276210764
WNA 3.15103981275
CNA 3.42469990056
ENA 2.01087566212
NCA 2.52038058236
SCA 3.52553646523
CAR 4.25304552979
NWS 2.32412707562
NSA 5.32681219602
NES 6.71262077465
SAM 6.21899407235
SWS 4.52956408695
SES 2.61100225609
SSA 3.61767147485
NEU 1.75688623539
WCE 2.65944052553
EEU 2.23112519759
MED 4.69099543538
SAH 3.04878654881
WAF 1.36234196566
CAF 2.24429478384
NEAF 1.71943513744
SEAF 3.97833011861
WSAF 5.16100823691
ESAF 5.52110496109
MDG 5.80477300647
RAR 0.891724961509
WSB 1.7748403112
ESB 1.16248335888
RFE 0.889968035885
WCA 2.01459187536
ECA 1.34210720907
TIB 2.40480613118
EAS 2.27370267585
ARP 2.60697450445
SAS 2.81633985665
SEA 3.32629058476
NAU 6.46774838636
CAU 5.82501202231
EAU 4.93365940418
SAU 4.61455592882
NZ 4.19938116239
EAN 2.42165457173
WAN 1.8894075423
model15_ssp585
GIC 2.39203532777
NWN 1.38778885502
NEN 2.14589335082
WNA 4.48348378271
CNA 4.04459732649
ENA 2.79460532129
NCA 4.54353506912
SCA 4.8

EAS 3.34714177045
ARP 1.89413249113
SAS 3.65728935875
SEA 4.19475348397
NAU 5.87502871577
CAU 6.06767502254
EAU 6.22083765624
SAU 6.41260444907
NZ 2.06337903321
EAN 2.50365824708
WAN 3.3621967815
model14_ssp126
GIC 2.03447489929
NWN 2.88721854131
NEN 2.25839064653
WNA 4.8805741757
CNA 3.85450089096
ENA 2.53510337517
NCA 4.93653987365
SCA 3.07761901723
CAR 4.6126003027
NWS 2.06636954365
NSA 3.84864563173
NES 4.93790161712
SAM 4.30945722148
SWS 4.54706028514
SES 3.74864060808
SSA 2.68831793601
NEU 3.32538119241
WCE 3.22400149386
EEU 2.91056703904
MED 5.13980387388
SAH 3.64403490184
WAF 2.35305977489
CAF 2.06520445137
NEAF 1.64427390208
SEAF 2.72829586363
WSAF 5.91480208149
ESAF 5.70305003911
MDG 5.59208577872
RAR 1.52031822608
WSB 2.54767098742
ESB 2.14030638642
RFE 1.93289243654
WCA 4.52496685452
ECA 2.72724782914
TIB 1.7131328298
EAS 3.68682820148
ARP 3.91617660158
SAS 3.04524929935
SEA 3.13890862632
NAU 3.77331182391
CAU 4.2317270465
EAU 4.09526703226
SAU 4.51192072933
NZ 4.2677968919

SEAF 2.16668308975
WSAF 6.00915689811
ESAF 5.26176596813
MDG 5.42030840036
RAR 1.42727003248
WSB 3.19792498147
ESB 1.59312569068
RFE 2.31262670715
WCA 3.075809194
ECA 1.92820194622
TIB 2.8715997978
EAS 2.58470892078
ARP 2.3373004735
SAS 3.99142053072
SEA 3.59468433987
NAU 5.17529738947
CAU 7.08522869606
EAU 5.59904484298
SAU 6.64897046726
NZ 3.31003615683
EAN 2.13728944809
WAN 2.30702077915
model3_ssp126
GIC 1.99759578711
NWN 1.7401264002
NEN 1.68885468194
WNA 3.25814449181
CNA 4.1040699092
ENA 3.24745979125
NCA 4.27589180445
SCA 3.88584428878
CAR 3.8888966531
NWS 4.41832139731
NSA 6.07679928109
NES 5.41670045316
SAM 5.24682902845
SWS 5.83087769882
SES 3.11430159272
SSA 4.42966273371
NEU 3.14243684503
WCE 3.78358990081
EEU 2.60335030248
MED 4.3193871206
SAH 3.64254630705
WAF 2.4241425447
CAF 2.95856365986
NEAF 2.73563672852
SEAF 4.4976331608
WSAF 6.77132566085
ESAF 6.95690829594
MDG 4.90556713024
RAR 1.64413349429
WSB 1.89442197058
ESB 1.77886102691
RFE 2.03221043082
WCA 3.18901269555


SAM 5.50022792331
SWS 6.24905873104
SES 3.7662208613
SSA 3.79823064249
NEU 1.34319784756
WCE 2.2393770363
EEU 1.15607751491
MED 4.64444833925
SAH 1.56022449898
WAF 1.48491612479
CAF 2.32712203837
NEAF 2.26647703903
SEAF 3.67483222533
WSAF 5.89204119042
ESAF 5.01887372885
MDG 5.01376947367
RAR 0.852775301086
WSB 0.844720492466
ESB 1.3322282481
RFE 1.11979253663
WCA 4.53642049937
ECA 1.90069409382
TIB 1.61323813543
EAS 2.31639274021
ARP 3.18801438991
SAS 2.64558355303
SEA 4.30146864524
NAU 4.09779233474
CAU 3.61354751418
EAU 3.29837088089
SAU 3.66032687328
NZ 1.82270060777
EAN 2.45552769613
WAN 2.9898307963
model31_ssp585
GIC 2.47891959368
NWN 2.53341207829
NEN 2.06172715909
WNA 2.76335350585
CNA 3.77190316517
ENA 3.13593034147
NCA 3.37249188728
SCA 5.35280031436
CAR 6.86305459359
NWS 5.36562883421
NSA 5.49895470611
NES 5.24249249821
SAM 4.81832234163
SWS 6.03877891065
SES 3.22105807475
SSA 3.46879697884
NEU 4.56383265474
WCE 5.19886845354
EEU 5.47211677268
MED 4.44085578847
SAH 3.040781

WNA 3.18483947847
CNA 4.056322405
ENA 2.87888683137
NCA 4.55098648572
SCA 4.13306517069
CAR 4.52591876713
NWS 3.3498823632
NSA 4.23479320927
NES 5.2651238039
SAM 3.88227415195
SWS 5.59905439064
SES 3.62524793824
SSA 3.95004243194
NEU 2.16568688113
WCE 2.33064367156
EEU 2.1341894211
MED 4.88625962323
SAH 1.30221938144
WAF 1.62094099008
CAF 2.03781925442
NEAF 2.02570309027
SEAF 3.80851322373
WSAF 4.57204931236
ESAF 5.61447661081
MDG 3.96044701047
RAR 1.01093654847
WSB 1.58368684062
ESB 2.24498106007
RFE 2.37028277294
WCA 3.39934732624
ECA 2.16706998323
TIB 1.62571545041
EAS 3.3652841693
ARP 3.7926268097
SAS 2.82053133501
SEA 4.22349767495
NAU 4.5732685101
CAU 4.28138209756
EAU 3.9324288161
SAU 4.92942145397
NZ 3.10954898529
EAN 3.2949429038
WAN 2.74953316161
model24_ssp585
GIC 3.19822175333
NWN 1.97052251306
NEN 2.14320525098
WNA 4.6112082931
CNA 4.37141666595
ENA 2.95300006262
NCA 4.19091136712
SCA 6.85816972464
CAR 8.51117103475
NWS 3.35578897582
NSA 8.64612189186
NES 5.23445835674
SAM

...

NSA 5.05515626547
NES 5.17311963732
SAM 5.68494533854
SWS 6.20445208886
SES 3.16931732072
SSA 3.9622312981
NEU 3.19540746674
WCE 5.23484828638
EEU 5.36707529969
MED 5.42929772575
SAH 3.19180159606
WAF 4.17189257453
CAF 1.34320108301
NEAF 1.00690645509
SEAF 1.65345842651
WSAF 3.46720997288
ESAF 3.34550441764
MDG 3.64953730411
RAR 2.03956977999
WSB 4.80214525058
ESB 2.87607563992
RFE 1.52174277669
WCA 3.05003320243
ECA 2.08948734119
TIB 4.64385979562
EAS 2.24640713294
ARP 2.17328944788
SAS 2.38363210658
SEA 4.08239792323
NAU 4.63266338592
CAU 4.16146106277
EAU 5.49048054279
SAU 3.76326323983
NZ 4.33498517783
EAN 2.34258164063
WAN 2.45121663452
model12
GIC 1.85704373079
NWN 1.92961498302
NEN 2.11070175636
WNA 3.77311862528
CNA 3.03226667823
ENA 1.82687139111
NCA 5.56921536667
SCA 7.73856194516
CAR 6.87413932521
NWS 4.02038971882
NSA 6.43270481831
NES 6.87652754501
SAM 3.8554355111
SWS 6.01429414405
SES 5.13328054922
SSA 4.92260234119
NEU 2.47283882446
WCE 4.26995108709
EEU 4.00028612693
M

CNA 4.6491597271
ENA 2.48098691242
NCA 5.1323449763
SCA 5.49230907592
CAR 6.47553647642
NWS 5.61630163217
NSA 6.80109930489
NES 7.91220964951
SAM 6.19558101168
SWS 5.36113765529
SES 4.07076616265
SSA 4.42637914203
NEU 3.96898625026
WCE 4.97657297499
EEU 3.7888613443
MED 5.95559903246
SAH 3.99516646136
WAF 2.89322019075
CAF 3.44306427179
NEAF 0.787043978293
SEAF 1.72369743052
WSAF 6.55766026684
ESAF 4.53336328379
MDG 5.04203277886
RAR 2.16923013236
WSB 3.28301236334
ESB 1.64279824267
RFE 1.41194997764
WCA 4.62113185425
ECA 1.39795266319
TIB 0.989057648459
EAS 3.23494433079
ARP 2.23818226811
SAS 2.39097124974
SEA 3.99373124719
NAU 3.0839192244
CAU 4.14451879938
EAU 4.17502552575
SAU 4.91150712334
NZ 3.04736323116
EAN 1.32698931115
WAN 1.80034258147
model30
GIC 3.43319319336
NWN 1.43523297154
NEN 1.60123024561
WNA 3.27949344678
CNA 2.92332735864
ENA 2.82686471355
NCA 3.78832057027
SCA 7.24317157371
CAR 6.3335017251
NWS 4.90778722779
NSA 6.32940530749
NES 5.37418109809
SAM 6.40909392823
SW

SAU 5.68964492575
NZ 3.25622831583
EAN 0.71042477132
WAN 1.36687490982
model27
GIC 2.05672788762
NWN 1.55766655985
NEN 1.33453413079
WNA 3.45948819077
CNA 3.78902286778
ENA 3.17116529737
NCA 6.65163830609
SCA 6.77924870425
CAR 7.40381050067
NWS 3.38637121614
NSA 6.30436493655
NES 4.03322844672
SAM 4.40201487616
SWS 5.77461910809
SES 3.79641158287
SSA 4.75029561341
NEU 1.22391255082
WCE 3.01838623638
EEU 2.63679316935
MED 7.78329300398
SAH 4.20200048666
WAF 9.12010282827
CAF 2.79885657789
NEAF 2.25315978073
SEAF 2.10006058951
WSAF 6.85862473438
ESAF 6.19090083577
MDG 5.45192968739
RAR 1.07055763712
WSB 3.00880182332
ESB 1.97087232749
RFE 0.90976902834
WCA 5.77315666617
ECA 3.40410643448
TIB 3.04492381597
EAS 3.24833482556
ARP 5.3656187255
SAS 2.99847602492
SEA 2.38674156437
NAU 1.38946263514
CAU 2.81035058455
EAU 2.3711606393
SAU 3.87010529048
NZ 4.11017146948
EAN 2.3087243621
WAN 2.3978183191
model24
GIC 1.55064999148
NWN 1.66767572625
NEN 1.71210846295
WNA 4.74754114835
CNA 4.91111986

TIB 2.6033171274
EAS 2.89818546482
ARP 1.91515326032
SAS 3.0172847212
SEA 3.18761520262
NAU 3.54283741545
CAU 4.21709740415
EAU 2.28215667869
SAU 5.96543220208
NZ 2.26552521849
EAN 1.10986736716
WAN 1.50830400029
model6
GIC 1.05220361946
NWN 0.885931385626
NEN 0.91081280957
WNA 3.85302188302
CNA 3.69979397374
ENA 1.96460938037
NCA 3.9055573882
SCA 6.40419492058
CAR 5.76362694875
NWS 3.15869651325
NSA 8.2115967982
NES 7.74060074547
SAM 7.44455143853
SWS 6.60939262189
SES 2.87839949286
SSA 5.0341191622
NEU 1.10442809589
WCE 3.63563847369
EEU 2.20724373911
MED 6.52292804256
SAH 3.05924117855
WAF 2.75646725656
CAF 1.86228854808
NEAF 1.51758970147
SEAF 2.39045164547
WSAF 6.33874326554
ESAF 6.29673995636
MDG 5.67131889385
RAR 0.348802088589
WSB 1.50279234806
ESB 0.901365393907
RFE 0.753370352891
WCA 2.03309250395
ECA 0.420037878097
TIB 2.57937603226
EAS 1.81069396935
ARP 1.99287792359
SAS 2.17999658746
SEA 4.55912487648
NAU 5.72172903603
CAU 4.73706768534
EAU 4.0021997431
SAU 4.61603938331
N

WSAF 7.29298258776
ESAF 7.24876452715
MDG 4.00017163371
RAR 1.04109381112
WSB 3.36395569748
ESB 1.17456175603
RFE 1.15101222012
WCA 3.8574084703
ECA 1.89080320626
TIB 2.72000048446
EAS 2.75976189689
ARP 4.24633564633
SAS 4.3548273369
SEA 5.67735372039
NAU 7.26874274101
CAU 7.66910504464
EAU 7.23283269931
SAU 7.72569958392
NZ 3.30913938309
EAN 0.842932619095
WAN 0.643491011514
model14
GIC 0.507095384491
NWN 0.439112544888
NEN 0.770200753671
WNA 2.41039725242
CNA 3.65728369521
ENA 2.27822122115
NCA 6.01373315752
SCA 6.26277924124
CAR 5.29468382855
NWS 4.78987581708
NSA 8.54962224052
NES 9.6797129474
SAM 8.72348728663
SWS 6.32403902352
SES 6.2595573033
SSA 6.13679751793
NEU 2.16224680992
WCE 1.4212215685
EEU 0.931492263644
MED 4.69092224926
SAH 1.92113135962
WAF 1.33062729807
CAF 2.62894447992
NEAF 0.483092116878
SEAF 1.93668895443
WSAF 7.13364866875
ESAF 6.69254026102
MDG 7.65210736007
RAR 0.583591952012
WSB 1.30223865565
ESB 0.527225859048
RFE 0.262641036225
WCA 2.82794277263
ECA 0.5597

### Compute the ensemble statistics

In [12]:
import numpy
quantiles_dict = dict()
for clim_period in regional_averages:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages[clim_period]:
        print clim_period, region_name
        quantiles_dict[clim_period][region_name] = dict()
        dat = np.array(regional_averages[clim_period][region_name])
        q10 = np.quantile(dat, 0.1)
        q50 = np.quantile(dat, 0.5)
        q90 = np.quantile(dat, 0.9)
        quantiles_dict[clim_period][region_name] = [q10, q50, q90]

GWL4.0 NEN
GWL4.0 NCA
GWL4.0 WAN
GWL4.0 EEU
GWL4.0 CAR
GWL4.0 WSAF
GWL4.0 CAU
GWL4.0 WSB
GWL4.0 NAU
GWL4.0 SWS
GWL4.0 NEU
GWL4.0 NES
GWL4.0 TIB
GWL4.0 WNA
GWL4.0 SES
GWL4.0 SAM
GWL4.0 NWS
GWL4.0 NSA
GWL4.0 SCA
GWL4.0 MDG
GWL4.0 CAF
GWL4.0 SEAF
GWL4.0 NZ
GWL4.0 RAR
GWL4.0 SEA
GWL4.0 SAS
GWL4.0 NWN
GWL4.0 EAU
GWL4.0 SAU
GWL4.0 ENA
GWL4.0 WAF
GWL4.0 WCA
GWL4.0 SAH
GWL4.0 GIC
GWL4.0 ESB
GWL4.0 RFE
GWL4.0 CNA
GWL4.0 EAS
GWL4.0 ARP
GWL4.0 MED
GWL4.0 ECA
GWL4.0 NEAF
GWL4.0 EAN
GWL4.0 WCE
GWL4.0 SSA
GWL4.0 ESAF
ssp126_farfut NEN
ssp126_farfut NCA
ssp126_farfut WAN
ssp126_farfut EEU
ssp126_farfut CAR
ssp126_farfut WSAF
ssp126_farfut CAU
ssp126_farfut WSB
ssp126_farfut NAU
ssp126_farfut SWS
ssp126_farfut NEU
ssp126_farfut NES
ssp126_farfut TIB
ssp126_farfut WNA
ssp126_farfut SES
ssp126_farfut SAM
ssp126_farfut NWS
ssp126_farfut NSA
ssp126_farfut SCA
ssp126_farfut MDG
ssp126_farfut CAF
ssp126_farfut SEAF
ssp126_farfut NZ
ssp126_farfut RAR
ssp126_farfut SEA
ssp126_farfut SAS
ssp126_farfut NWN
ssp1

In [13]:
quantiles_dict[clim_period][region_name]

[4.03703536580345, 4.676718646300794, 5.90306699493598]

### Save in a json file

In [14]:
import json
ensemble = 'CMIP6'
outfilename = '/home/jservon/Chapter12_IPCC/data/Figure_S12.3/'+ensemble+'_DF6_AR6_regional_averages.json'
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

## CMIP5

In [9]:
# -- Scenarios - timelines
# ---------------------------------------------------------------------------

exp_list = [
    # -- Baseline (ssp126 and ssp585 are the same files)
    dict(experiment='rcp85',
         clim_period = 'hist'),    
    # -- Mid term
    dict(experiment='rcp85',
         clim_period = 'midfut'),
    dict(experiment='rcp26',
         clim_period = 'midfut'),
    # -- Late term
    dict(experiment='rcp85',
         clim_period = 'farfut'),
    dict(experiment='rcp26',
         clim_period = 'farfut'),
]

# -- Loop on the scenarios
ens_exp_dict = dict()
for exp in exp_list:
    #
    # -- Experiment and period
    experiment = exp['experiment']
    clim_period = exp['clim_period']
    
    # -- Create ensemble object for the scenario
    req_exp = ds(project='DF_cmip5_ch12',
                 experiment = experiment,
                 clim_period = clim_period,
                 member = '*'
                )
    ens_exp = req_exp.explore('ensemble')
    
    # -- Climatologies
    ens_exp_dict[experiment+'_'+clim_period] = ens_exp
#


# -- Global Warming Levels
# ---------------------------------------------------------------------------
ens_GWL_dict = dict()

# -- Loop on the GWLs
for GWL in ['GWL1.5','GWL2.0','GWL3.0','GWL4.0']:
    #
    if GWL in ['GWL1.5','GWL2.0']:
        # -- Create ensemble object for the scenario
        req_rcp26 = ds(project='DF_cmip5_GWL_ch12',
                     experiment = 'rcp26',
                     GWL = GWL,
                     member = '*'
                    )
        ens_rcp26 = req_rcp26.explore('ensemble')

        # -- Create ensemble object for the scenario
        req_rcp85 = ds(project='DF_cmip5_GWL_ch12',
                     experiment = 'rcp85',
                     GWL = GWL,
                     member = '*'
                    )
        ens_rcp85 = req_rcp85.explore('ensemble')
        #
        # -- Merge ensembles
        GWL_ens = merge_climaf_ensembles([add_prefix_suffix_to_ens_req(ens_rcp26,suffix='_rcp26'),
                                          add_prefix_suffix_to_ens_req(ens_rcp85,suffix='_rcp85')])
    else:
        # -- Create ensemble object for the scenario
        req_rcp85 = ds(project='DF_cmip5_GWL_ch12',
                     experiment = 'rcp85',
                     GWL = GWL,
                     member = '*'
                    )
        GWL_ens = req_rcp85.explore('ensemble')
        
    # -- Climatologies
    ens_GWL_dict[GWL] = GWL_ens
    


In [10]:
regional_averages_CMIP5 = dict()

# -- Loop on experiments / horizons
for ens_exp in ens_exp_dict:
    print ens_exp
    regional_averages_CMIP5[ens_exp] = dict()
    # -- Loop on the members of each ensemble
    for mem in ens_exp_dict[ens_exp]:
        print mem
        # -- Compute the averages for each AR6 region thanks to regionmask
        tmp = average_over_AR6_region(cfile(ens_exp_dict[ens_exp][mem]), 'DF6', 'all')
        region_names = list(tmp.abbrevs)
        ttmp = average_over_AR6_region(cfile(ens_exp_dict[ens_exp][mem]), 'DF6', region_names)
        for tmp_region_name in region_names:
            region_name = str(tmp_region_name.values)
            region_value = float(ttmp[list(tmp.abbrevs).index(region_name)])
            print region_name, region_value
            if region_name not in regional_averages_CMIP5[ens_exp]:
                regional_averages_CMIP5[ens_exp][region_name] = [region_value]
            else:
                regional_averages_CMIP5[ens_exp][region_name].append(region_value)
#

for GWL in ens_GWL_dict :
    print GWL
    regional_averages_CMIP5[GWL] = dict()
    # -- Loop on the members of each ensemble
    for mem in ens_GWL_dict[GWL]:
        print mem
        # -- Compute the averages for each AR6 region thanks to regionmask
        tmp = average_over_AR6_region(cfile(ens_GWL_dict[GWL][mem]), 'DF6', 'all')
        region_names = list(tmp.abbrevs)
        ttmp = average_over_AR6_region(cfile(ens_GWL_dict[GWL][mem]), 'DF6', region_names)
        for tmp_region_name in region_names:
            region_name = str(tmp_region_name.values)
            region_value = float(ttmp[list(tmp.abbrevs).index(region_name)])
            print region_name, region_value
            if region_name not in regional_averages_CMIP5[GWL]:
                regional_averages_CMIP5[GWL][region_name] = [region_value]
            else:
                regional_averages_CMIP5[GWL][region_name].append(region_value)


rcp26_farfut
model9
GIC 3.30030941714
NWN 1.99351213281
NEN 2.67290824939
WNA 4.99279261414
CNA 3.45389393505
ENA 3.03603152518
NCA 4.92909564505
SCA 4.89391820468
CAR 4.386212989
NWS 3.86647333643
NSA 4.75611084354
NES 5.2679799924
SAM 5.40831582804
SWS 5.02041863378
SES 3.78487497882
SSA 3.86142970132
NEU 2.73243343594
WCE 3.41821627291
EEU 3.70401481268
MED 4.45764446307
SAH 2.69143086386
WAF 4.33043685134
CAF 3.34751289528
NEAF 3.6895650914
SEAF 4.03390958256
WSAF 6.105051401
ESAF 5.01176511068
MDG 4.30440677428
RAR 1.71376236012
WSB 2.87248491727
ESB 1.89320425759
RFE 2.90478108
WCA 4.19354797188
ECA 3.05507439802
TIB 4.08644289103
EAS 4.2553367594
ARP 3.70160675351
SAS 5.14309616878
SEA 3.25261737459
NAU 4.23341791898
CAU 4.56064663169
EAU 4.61916821406
SAU 5.4880813624
NZ 3.03985545533
EAN 3.10469945931
WAN 2.87617034213
model8
GIC 3.86100759797
NWN 2.09362345726
NEN 1.51626283751
WNA 4.33876962432
CNA 3.12348162735
ENA 2.59820582694
NCA 3.26173714508
SCA 3.02315605957
CAR 3.120

SEA 4.1975107627
NAU 5.31386667548
CAU 7.14687277141
EAU 7.33018227214
SAU 6.20272390016
NZ 2.82296652831
EAN 2.83919264591
WAN 3.41622208524
model13
GIC 1.6586115055
NWN 1.96806028055
NEN 1.68667541404
WNA 3.06323599495
CNA 3.73563528651
ENA 1.95193596405
NCA 5.55997527542
SCA 3.07443855245
CAR 3.00000012818
NWS 2.10652074264
NSA 4.83855919086
NES 4.72181298117
SAM 2.89713107193
SWS 5.48185989814
SES 3.91021989135
SSA 4.59475829631
NEU 3.23840211124
WCE 2.79885724044
EEU 2.5922163751
MED 3.4904454882
SAH 2.198145557
WAF 2.73197271566
CAF 3.2124911744
NEAF 2.65847974922
SEAF 3.50712288983
WSAF 3.77847397158
ESAF 4.07612497166
MDG 4.80176617585
RAR 2.283799721
WSB 1.60017822464
ESB 1.33316128841
RFE 1.3775065363
WCA 3.84646010604
ECA 2.96187968568
TIB 3.3513624387
EAS 2.67925862727
ARP 3.43391726103
SAS 3.84335106118
SEA 3.31246609646
NAU 5.04242531688
CAU 5.53609213009
EAU 4.94062081663
SAU 5.37282232835
NZ 4.92370026033
EAN 3.34220145733
WAN 3.95704842948
model12
GIC 2.28340521547
NWN

ESB 1.46620983346
RFE 1.95526411015
WCA 4.2183551372
ECA 1.95769968206
TIB 1.42300299118
EAS 2.54307973887
ARP 4.6089826416
SAS 3.00162590395
SEA 3.57423710754
NAU 4.4721998857
CAU 4.61640139561
EAU 4.83919420247
SAU 4.69441481296
NZ 3.84928820644
EAN 1.5512118061
WAN 2.49124710056
model20
GIC 2.28632746194
NWN 1.79421985641
NEN 2.03279132134
WNA 4.29593923892
CNA 2.24161185773
ENA 2.46128196231
NCA 4.64559742499
SCA 4.42516216925
CAR 4.37436931473
NWS 2.9024068005
NSA 4.19834752481
NES 4.31164004464
SAM 4.0765684534
SWS 4.95817138432
SES 4.32031568984
SSA 3.12746563803
NEU 4.05649671744
WCE 2.3253588874
EEU 2.38656546732
MED 3.60961820988
SAH 6.71937041503
WAF 5.69190935719
CAF 4.53807693738
NEAF 3.57614575962
SEAF 3.38091012892
WSAF 4.64499111605
ESAF 4.8218785679
MDG 5.00673392659
RAR 2.4709227967
WSB 1.87880834489
ESB 1.89494669236
RFE 2.1844807844
WCA 3.47237121812
ECA 2.84409973876
TIB 3.03691764653
EAS 3.83084242529
ARP 4.70274402499
SAS 3.83344789517
SEA 3.95455626962
NAU 5.709

NEAF 4.23034209895
SEAF 6.20026261348
WSAF 6.6516970644
ESAF 6.02267815329
MDG 6.97953965985
RAR 1.56989514228
WSB 2.34963749899
ESB 1.71274152166
RFE 1.81108445344
WCA 5.55244604121
ECA 1.6223260271
TIB 2.14749023062
EAS 3.31723546485
ARP 4.81014561657
SAS 4.36416037869
SEA 3.47895651439
NAU 6.08945441908
CAU 6.44844918005
EAU 5.49275111982
SAU 6.23865663298
NZ 3.96248929494
EAN 4.424852291
WAN 3.28184870364
model5
GIC 1.32667418679
NWN 0.647670862827
NEN 0.814324560006
WNA 3.10569930802
CNA 1.52606059854
ENA 1.18828124912
NCA 6.44168827318
SCA 2.89046261872
CAR 5.5239922206
NWS 7.2335055419
NSA 7.23870659079
NES 7.46694940151
SAM 9.86455479758
SWS 7.81505412304
SES 7.35050727779
SSA 6.98405040533
NEU 1.04176975835
WCE 3.83812281391
EEU 1.81225693913
MED 8.02690019584
SAH 3.26675097472
WAF 3.14326601194
CAF 5.40547274616
NEAF 2.80202059279
SEAF 5.31744376454
WSAF 6.93525006131
ESAF 7.31370687338
MDG 7.37574874776
RAR 0.0910583416811
WSB 0.853818501209
ESB 0.768338313449
RFE 0.90775602

EEU 1.9134338882
MED 5.71592964211
SAH 2.8553028674
WAF 4.11856930433
CAF 2.09599329424
NEAF 3.31536377566
SEAF 3.30479464027
WSAF 5.86004985535
ESAF 6.41079530364
MDG 4.95277694936
RAR 1.76022663764
WSB 2.37722154595
ESB 1.44792722664
RFE 1.63875684499
WCA 5.01896977479
ECA 2.741237858
TIB 1.8229827433
EAS 3.1325736189
ARP 4.57164315461
SAS 3.08485948046
SEA 3.35334263145
NAU 4.2051517277
CAU 5.59663942975
EAU 6.27128045884
SAU 5.38216375374
NZ 1.55109504639
EAN 2.43438795436
WAN 2.6777801392
model22
GIC 1.09108153684
NWN 1.90392752853
NEN 2.05934083599
WNA 4.6870232482
CNA 4.74718390878
ENA 2.56697948467
NCA 7.69045701106
SCA 7.69029440991
CAR 7.01841473445
NWS 1.9192873992
NSA 5.75387139473
NES 5.06444641694
SAM 3.67764234187
SWS 6.34531109809
SES 4.93181685877
SSA 5.50442001862
NEU 3.01122825153
WCE 3.29066888159
EEU 3.37620807965
MED 5.38000197659
SAH 3.2161756904
WAF 3.97099247269
CAF 2.87211068321
NEAF 2.99696448283
SEAF 4.05011180516
WSAF 4.2660223494
ESAF 3.18998983942
MDG 4.6

...

WAF 4.01619660345
CAF 3.05560564113
NEAF 3.84852881713
SEAF 6.24200032444
WSAF 5.97494125131
ESAF 5.55642119304
MDG 5.9848457482
RAR 0.634635610338
WSB 1.40716842228
ESB 1.46532565786
RFE 0.882458261321
WCA 5.71570139295
ECA 2.71816583144
TIB 3.85593300796
EAS 3.77105098546
ARP 3.61173993859
SAS 5.29139545183
SEA 3.19132505382
NAU 4.16482874045
CAU 5.95982816915
EAU 4.82518205622
SAU 6.58727787289
NZ 3.41552826908
EAN 1.63908720383
WAN 1.99543371181
model7
GIC 2.81083050732
NWN 1.36856791015
NEN 1.06342816115
WNA 3.42860587637
CNA 3.08561422575
ENA 2.00395726265
NCA 5.9556726895
SCA 3.71940922141
CAR 6.36779863908
NWS 2.18714571
NSA 3.31857560753
NES 6.87980404869
SAM 6.68169884031
SWS 8.9924148766
SES 6.51111891082
SSA 7.76778432743
NEU 1.37248810302
WCE 4.3391144807
EEU 2.70306851991
MED 7.54155913057
SAH 3.98722316981
WAF 4.01890266733
CAF 4.96257885404
NEAF 3.05921296516
SEAF 3.22525017214
WSAF 7.36541874828
ESAF 7.94394839332
MDG 8.27389416044
RAR 0.81627476874
WSB 0.928742507087


SAM 5.22233521154
SWS 8.15781919814
SES 3.55368732146
SSA 6.29781196711
NEU 2.24857481555
WCE 5.74828795705
EEU 3.91521074109
MED 7.27320609588
SAH 2.89537806847
WAF 4.018593211
CAF 2.21818689272
NEAF 2.46415877488
SEAF 2.84692238291
WSAF 6.54042267112
ESAF 6.90919044009
MDG 6.92345912716
RAR 0.916237443858
WSB 2.74137864837
ESB 1.65287194579
RFE 1.77605174963
WCA 5.75289506917
ECA 2.50352683513
TIB 4.40224730888
EAS 3.6035041439
ARP 4.03108008796
SAS 5.17231294625
SEA 2.4563598315
NAU 5.04252662267
CAU 5.52811263105
EAU 6.75436719023
SAU 6.09649915756
NZ 4.39410301647
EAN 2.40038975655
WAN 2.24234497942
model15
GIC 1.50472276227
NWN 1.55144225261
NEN 1.10599044631
WNA 3.58441854515
CNA 5.45258788747
ENA 3.19293497733
NCA 6.50931575974
SCA 4.5796603026
CAR 3.74262670392
NWS 1.3583138666
NSA 5.17421288921
NES 6.38796889134
SAM 4.28102471835
SWS 7.60891147267
SES 3.34529106915
SSA 7.32534555294
NEU 2.11148311757
WCE 2.0213919448
EEU 2.89178735522
MED 5.68683195465
SAH 3.10860886386
WAF 4

WNA 3.63187210846
CNA 3.17764477958
ENA 2.52396680203
NCA 6.2182569774
SCA 7.96918947404
CAR 8.00672372053
NWS 3.32005665255
NSA 5.53784631452
NES 5.53814837542
SAM 4.58680538163
SWS 5.2636214102
SES 3.1089344957
SSA 4.40504076015
NEU 2.95000432833
WCE 4.27678138434
EEU 2.84321638621
MED 6.27382896965
SAH 3.01677975226
WAF 3.28065584078
CAF 1.14446330381
NEAF 1.3020298687
SEAF 2.35420199242
WSAF 4.55253556633
ESAF 3.91278160425
MDG 3.9797983758
RAR 0.633418751173
WSB 2.56806981696
ESB 1.57501128873
RFE 1.28426736726
WCA 2.51198684878
ECA 2.10317012683
TIB 1.78886859218
EAS 3.20999740502
ARP 3.15581269902
SAS 5.03861207934
SEA 2.41428001356
NAU 2.47718544377
CAU 5.33044210777
EAU 2.96391537673
SAU 5.55408657689
NZ 3.9322711815
EAN 0.662987071767
WAN 1.95340257653
model21
GIC 0.868555262169
NWN 1.02573870222
NEN 1.11763512793
WNA 4.00523863494
CNA 3.9207586659
ENA 2.1403539745
NCA 5.11276258148
SCA 4.21413731899
CAR 5.11523882244
NWS 1.63136066296
NSA 4.22091641484
NES 5.91730716097
SAM 

SAU 7.04572226372
NZ 3.31895637889
EAN 0.728622091506
WAN 1.61764161338
model6
GIC 0.426070443698
NWN 1.56006795609
NEN 1.64241586474
WNA 5.50940058929
CNA 3.80602652311
ENA 3.35829412277
NCA 6.0742505691
SCA 5.66894676545
CAR 6.13380608906
NWS 3.05810454341
NSA 8.21353059382
NES 7.12915246881
SAM 7.47296673247
SWS 5.96028617712
SES 3.00007344374
SSA 6.37939481538
NEU 3.61977616318
WCE 5.80374515421
EEU 3.28044725463
MED 7.81223213161
SAH 4.48941149592
WAF 2.9363128278
CAF 2.71478283268
NEAF 2.51313957827
SEAF 6.72573388213
WSAF 7.30260674128
ESAF 7.91136070323
MDG 9.11350100554
RAR 0.230402911701
WSB 1.6016200378
ESB 1.3846924905
RFE 0.742746689758
WCA 4.86372839533
ECA 1.90867701718
TIB 3.06338932413
EAS 2.57284054197
ARP 4.67346860479
SAS 2.98726295211
SEA 5.16368786721
NAU 5.20403150141
CAU 6.28455154561
EAU 6.1416976473
SAU 7.15930271877
NZ 5.6200235251
EAN 0.946585755377
WAN 0.778824201535
model5
GIC 0.553092173579
NWN 1.06768456483
NEN 1.24345070119
WNA 5.03961514602
CNA 4.58471

WCA 5.53945687006
ECA 2.38856176282
TIB 2.82593333615
EAS 2.96217822718
ARP 2.22297963499
SAS 1.74199707692
SEA 4.04647175811
NAU 3.08414125873
CAU 5.59243684078
EAU 4.26457215315
SAU 6.43048938935
NZ 3.91813719568
EAN 0.787565894971
WAN 0.76414588349
model14
GIC 0.392499233103
NWN 0.479908029332
NEN 0.575330636445
WNA 5.38945574681
CNA 3.75081380504
ENA 3.71072152341
NCA 5.80600279407
SCA 5.92803679322
CAR 6.48830981924
NWS 1.76496068981
NSA 4.37953430306
NES 6.86720226217
SAM 3.14811359395
SWS 6.40595714474
SES 3.9540576863
SSA 6.06796622284
NEU 1.6620131623
WCE 4.97600186563
EEU 1.43452458901
MED 8.43935148348
SAH 6.31933690819
WAF 5.01954702361
CAF 4.19805976723
NEAF 2.25133598064
SEAF 2.70159433798
WSAF 6.9004518552
ESAF 6.9255247473
MDG 4.8504592247
RAR 0.736225498295
WSB 1.39754332296
ESB 0.826764384891
RFE 0.881543933171
WCA 7.01429604154
ECA 2.68133976602
TIB 2.80094669798
EAS 2.49977405912
ARP 4.57431978924
SAS 2.17295488652
SEA 3.28355460512
NAU 3.79154645398
CAU 4.446859450

In [11]:
quantiles_dict = dict()
for clim_period in regional_averages_CMIP5:
    quantiles_dict[clim_period] = dict()
    for region_name in regional_averages_CMIP5[clim_period]:
        print clim_period, region_name
        quantiles_dict[clim_period][region_name] = dict()
        dat = np.array(regional_averages_CMIP5[clim_period][region_name])
        q10 = np.quantile(dat, 0.1)
        q50 = np.quantile(dat, 0.5)
        q90 = np.quantile(dat, 0.9)
        quantiles_dict[clim_period][region_name] = [q10, q50, q90]

rcp85_farfut NEN
rcp85_farfut NCA
rcp85_farfut WAN
rcp85_farfut EEU
rcp85_farfut CAR
rcp85_farfut WSAF
rcp85_farfut CAU
rcp85_farfut WSB
rcp85_farfut NAU
rcp85_farfut SWS
rcp85_farfut NEU
rcp85_farfut NES
rcp85_farfut TIB
rcp85_farfut WNA
rcp85_farfut SES
rcp85_farfut SAM
rcp85_farfut NWS
rcp85_farfut NSA
rcp85_farfut SCA
rcp85_farfut MDG
rcp85_farfut CAF
rcp85_farfut SEAF
rcp85_farfut NZ
rcp85_farfut RAR
rcp85_farfut SEA
rcp85_farfut SAS
rcp85_farfut NWN
rcp85_farfut EAU
rcp85_farfut SAU
rcp85_farfut ENA
rcp85_farfut WAF
rcp85_farfut WCA
rcp85_farfut SAH
rcp85_farfut GIC
rcp85_farfut ESB
rcp85_farfut RFE
rcp85_farfut CNA
rcp85_farfut EAS
rcp85_farfut ARP
rcp85_farfut MED
rcp85_farfut ECA
rcp85_farfut NEAF
rcp85_farfut EAN
rcp85_farfut WCE
rcp85_farfut SSA
rcp85_farfut ESAF
GWL4.0 NEN
GWL4.0 NCA
GWL4.0 WAN
GWL4.0 EEU
GWL4.0 CAR
GWL4.0 WSAF
GWL4.0 CAU
GWL4.0 WSB
GWL4.0 NAU
GWL4.0 SWS
GWL4.0 NEU
GWL4.0 NES
GWL4.0 TIB
GWL4.0 WNA
GWL4.0 SES
GWL4.0 SAM
GWL4.0 NWS
GWL4.0 NSA
GWL4.0 SCA
GWL4.

In [12]:
import json
ensemble = 'CMIP5'
outfilename = '/home/jservon/Chapter12_IPCC/data/Figure_S12.3/'+ensemble+'_DF6_AR6_regional_averages.json'
#print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

## CORDEX, all domains

In [5]:
pattern = '/data/jservon/IPCC/DF/individual_models/CORDEX/${CORDEX_domain}/${CORDEX_domain}_${experiment}_DF6_${clim_period}_${member}.nc'
pattern_EUR = '/data/jservon/IPCC/DF/CORDEX/${CORDEX_domain}/${experiment}/pr_mon_${member}_OK_SPI6_sel_SPELL_${clim_period}.nc'
cproject('DF_cordex_ch12','experiment','clim_period',('period','fx'),'CORDEX_domain','member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cordex_ch12', url=[pattern,pattern_EUR])

pattern_GWL = '/data/jservon/IPCC/DF/individual_models/CORDEX/${CORDEX_domain}/${CORDEX_domain}_${experiment}_DF6_${clim_period}_${member}.nc'
pattern_EUR_GWL = '/data/jservon/IPCC/DF/CORDEX/${CORDEX_domain}/GWLs/pr_mon_${member}_OK_SPI6_sel_SPELL_${GWL}deg.nc'
cproject('DF_cordex_GWL_ch12','experiment','clim_period',('period','fx'),'CORDEX_domain','member', ('variable','DF6'), ensemble=['member'], separator='%')
dataloc(project='DF_cordex_GWL_ch12', url=[pattern_GWL,pattern_EUR_GWL])

clog('critical')
#thres = 'Exceed'+wthres
# -- Scenarios - timelines
# ---------------------------------------------------------------------------
CORDEX_domains = [
    'AFR-22',
    'AUS-22',
    'CAM-22',
    'EAS-22',
    'EUR-11',
    'NAM-22',
    'SAM-22',
    'SEA-22',
    'WAS-22'
    
]



exp_list = [
    # -- Baseline (ssp126 and ssp585 are the same files)
    dict(experiment='rcp85',
         clim_period = 'hist'),    
    # -- Mid term
    dict(experiment='rcp85',
         clim_period = 'midfut'),
    dict(experiment='rcp26',
         clim_period = 'midfut'),
    # -- Late term
    dict(experiment='rcp85',
         clim_period = 'farfut'),
    dict(experiment='rcp26',
         clim_period = 'farfut'),
]

ens_exp_dict_CORDEX = dict()
ens_GWL_dict_CORDEX = dict()

#for CORDEX_domain in ['AFR-22']:
for CORDEX_domain in CORDEX_domains:
    #
    # -- We use the first three letters of the CORDEX_domain to identify the region
    short_CORDEX_domain = CORDEX_domain[0:3]

    # -- Loop on the scenarios
    ens_exp_dict_CORDEX[short_CORDEX_domain] = dict()
    for exp in exp_list:
        #
        # -- Experiment and period
        experiment = exp['experiment']
        clim_period = exp['clim_period']

        # -- Create ensemble object for the scenario
        req_exp = ds(project='DF_cordex_ch12',
                     experiment = experiment,
                     #realization='r1i1p1',
                     clim_period = clim_period,
                     CORDEX_domain = CORDEX_domain,
                     member = '*'
                    )
        #try:
        ens_exp = add_prefix_suffix_to_ens_req(req_exp.explore('ensemble'), suffix='_'+CORDEX_domain)
        #
        # -- Climatologies
        # If there is already a short_CORDEX_domain in the results, we merge the results of the new one
        if experiment+'_'+clim_period in ens_exp_dict_CORDEX[short_CORDEX_domain]:
            #res_copy = ens_exp_dict_CORDEX[short_CORDEX_domain][experiment+'_'+period].copy()
            #final_ens = merge_climaf_ensembles([res_copy, clim_average(ens_exp, 'ANM')])
            ens_exp_dict_CORDEX[short_CORDEX_domain][experiment+'_'+clim_period].update( clim_average(ens_exp, 'ANM') )
        else:
            ens_exp_dict_CORDEX[short_CORDEX_domain][experiment+'_'+clim_period] = clim_average(ens_exp, 'ANM')
            #final_ens = clim_average(ens_exp, 'ANM')
        #ens_exp_dict_CORDEX[short_CORDEX_domain][experiment+'_'+period] = final_ens
        #except:
        #print 'no data found for ', req_exp
    #

    # -- Global Warming Levels
    # ---------------------------------------------------------------------------
    ens_GWL_dict_CORDEX[short_CORDEX_domain] = dict()

    # -- Loop on the GWLs
    for GWL in ['GWL1.5','GWL2.0','GWL4.0']:
        #
        if GWL in ['GWL1.5']:
            # -- Create ensemble object for the scenario
            req_rcp26 = ds(project='DF_cordex_ch12',
                         experiment = 'rcp26',
                         clim_period = GWL,
                         CORDEX_domain = CORDEX_domain,
                         member = '*'
                        )
            try:
                ens_rcp26 = req_rcp26.explore('ensemble')
            except:
                print 'No data found for ',req_rcp26
                ens_rcp26 = cens(dict())

            # -- Create ensemble object for the scenario
            req_rcp85 = ds(project='DF_cordex_ch12',
                         experiment = 'rcp85',
                         clim_period = GWL,
                         CORDEX_domain = CORDEX_domain,
                         member = '*'
                        )
            try:
                ens_rcp85 = req_rcp85.explore('ensemble')
            except:
                print 'No data found for ',req_rcp85
                ens_rcp85 = cens(dict())
            #
            # -- Merge ensembles
            GWL_ens = merge_climaf_ensembles([add_prefix_suffix_to_ens_req(ens_rcp26,suffix='_rcp26'),
                                              add_prefix_suffix_to_ens_req(ens_rcp85,suffix='_rcp85')])
        else:
            # -- Create ensemble object for the scenario
            req_rcp85 = ds(project='DF_cordex_ch12',
                         experiment = 'rcp85',
                         #  realization='r1i1p1',
                         clim_period = GWL,
                         CORDEX_domain = CORDEX_domain,
                         member = '*'
                        )
            GWL_ens = req_rcp85.explore('ensemble')
        #
        # -- Rename the members with the CORDEX domain for future merge
        renamed_GWL_ens = add_prefix_suffix_to_ens_req(GWL_ens, suffix='_'+CORDEX_domain)
        #
        # -- if GWL in ens_GWL_dict_CORDEX[short_CORDEX_domain]:
        if GWL in ens_GWL_dict_CORDEX[short_CORDEX_domain]:
            res_copy = ens_GWL_dict_CORDEX[short_CORDEX_domain][GWL].copy()
            final_ens = merge_climaf_ensembles([res_copy, clim_average(renamed_GWL_ens, 'ANM')])
            # We merge the new one with the old one
        else:
            # We just add the new one
            final_ens = clim_average(renamed_GWL_ens, 'ANM')
        #
        # -- Climatologies
        ens_GWL_dict_CORDEX[short_CORDEX_domain][GWL] = final_ens
    #

In [6]:
def average_over_AR6_region(filename, variable, region_name):

    # -- AR6 regions
    #ar6_all = regionmask.defined_regions.ar6.all
    # -- Get the regions
    ar6_land = regionmask.defined_regions.ar6.land

    #ax = ar6_all.plot()
    # -- Get land/sea mask (generic)
    land_110 = regionmask.defined_regions.natural_earth.land_110

    # -- Get data
    ds = xr.open_dataset(filename, decode_times=False)
    dat = ds[variable]
    dat.values = np.array(dat.values, dtype=np.float32)

    # -- Mask the data
    mask_3D = ar6_land.mask_3D(dat) # AR6 mask
    land_mask = land_110.mask_3D(dat) # Land sea mask
    mask_lsm = mask_3D * land_mask.squeeze(drop=True) # Combine the two

    weights = np.cos(np.deg2rad(dat.lat))
    
    if region_name=='all':
        tmp = weighted_mean(dat, mask_lsm * weights, ("lon", "lat"))
        tmp['abbrevs'] = mask_3D.abbrevs
        return tmp
    else:
        # -- Case dat is has time dim
        if 'time' in dat.dims:
            matlat = np.mean(dat.values, axis=dat.dims.index('time')) * 0
        else:
            matlat = dat.values * 0

        if dat.dims.index('lat')<dat.dims.index('lon'):
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[:,i] = dat.lat
        else:
            for i in range(0,dat.shape[dat.dims.index('lon')]):
                matlat[i,:] = dat.lat
    
        weights = np.cos(np.deg2rad(matlat))

        if isinstance(region_name, list):
            res = list()
            for region in region_name:
                region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region))
                dat_region = dat.where(region_mask)
                weights_region = np.where(region_mask, weights, float("nan"))
                #weights_region = weights.where(region_mask)
                res.append( weighted_mean(dat_region, region_mask*weights_region, ("lon","lat")) )
            return res
        else:
            region_mask = mask_lsm.isel(region=list(mask_3D.abbrevs).index(region_name))
            dat_region = dat.where(region_mask)
            weights_region = np.where(region_mask, weights, float("nan"))
            #weights_region = weights.where(region_mask)            
            return weighted_mean(dat_region, region_mask*weights_region, ("lon","lat"))
#    

#
# -- Loop on the short CORDEX domains
# -- short_CORDEX_domains = ['AFR','EAS',...]
regional_averages_CORDEX = dict()

# -- Merge the dictionary with scenarios and the one with the GWLs
#ens_CORDEX = ens_exp_dict_CORDEX.copy()
#ens_CORDEX.update(ens_GWL_dict_CORDEX)

AR6regions_by_CORDEX_domain = dict(
    AUS = ['NAU','CAU','EAU','SAU','NZ'],
    SEA = ['SEA'],
    WAS = ['ARP','SAS','WCA'],#,'TIB'],
    EAS = ['TIB','ECA','EAS'],#'SAS'
    CAM = ['NSA','SCA','CAR'],
    SAM = ['NWS','NSA','SAM','NES','SWS','SES','SSA'],
    NAM = ['NWN','NEN','WNA','CNA','ENA','NCA'],#,'GAP'],
    EUR = ['MED','WCE','NEU'],
    AFR = ['WAF','SAH','CAF','WSAF','ESAF','MDG','SEAF','NEAF','ARP']
)

for ens_CORDEX in [ens_exp_dict_CORDEX, ens_GWL_dict_CORDEX]:
    for short_CORDEX_domain in ens_CORDEX:
        print short_CORDEX_domain
        if short_CORDEX_domain not in regional_averages_CORDEX:
            regional_averages_CORDEX[short_CORDEX_domain] = dict()

        # -- Loop on experiments / horizons
        for ens_exp in ens_CORDEX[short_CORDEX_domain]:
            if ens_exp not in ['3.0K']:
                print ens_exp
                regional_averages_CORDEX[short_CORDEX_domain][ens_exp] = dict()
                # -- Loop on the members of each ensemble
                for mem in ens_CORDEX[short_CORDEX_domain][ens_exp]:
                    #if 'CCCma-CanESM2--UCAN-WRF341I' not in mem:
                    print mem
                    # -- Compute the averages for each AR6 region thanks to regionmask
                    # --> We regrid on a 0.25° grid with conservative regridding
                    tmp = average_over_AR6_region(
                                cfile(regridn(ens_CORDEX[short_CORDEX_domain][ens_exp][mem], cdogrid='r1440x720', option='remapbil')),
                                'DF6', 'all')
                    region_names = tmp.abbrevs
                    #for tmp_region_name in region_names:
                    for tmp_region_name in AR6regions_by_CORDEX_domain[short_CORDEX_domain]:
                        #region_name = str(tmp_region_name.values)
                        region_name = tmp_region_name
                        print region_name
                        region_value = float(tmp.sel(region=list(tmp.abbrevs).index(region_name)).values)
                        region_value = float(tmp.sel(region=list(tmp.abbrevs).index(region_name)).values)
                        if region_name not in regional_averages_CORDEX[short_CORDEX_domain][ens_exp]:
                            regional_averages_CORDEX[short_CORDEX_domain][ens_exp][region_name] = [region_value]
                        else:
                            regional_averages_CORDEX[short_CORDEX_domain][ens_exp][region_name].append(region_value)




SEA
rcp26_farfut
model3_SEA-22
SEA
model1_SEA-22
SEA
model2_SEA-22
SEA
model0_SEA-22
SEA
model5_SEA-22
SEA
model4_SEA-22
SEA
rcp85_midfut
model8_SEA-22
SEA
model3_SEA-22
SEA
model0_SEA-22
SEA
model5_SEA-22
SEA
model6_SEA-22
SEA
model4_SEA-22
SEA
model1_SEA-22
SEA
model2_SEA-22
SEA
model7_SEA-22
SEA
rcp85_farfut
model8_SEA-22
SEA
model3_SEA-22
SEA
model0_SEA-22
SEA
model5_SEA-22
SEA
model6_SEA-22
SEA
model4_SEA-22
SEA
model1_SEA-22
SEA
model2_SEA-22
SEA
model7_SEA-22
SEA
rcp26_midfut
model3_SEA-22
SEA
model1_SEA-22
SEA
model2_SEA-22
SEA
model0_SEA-22
SEA
model5_SEA-22
SEA
model4_SEA-22
SEA
rcp85_hist
model8_SEA-22
SEA
model3_SEA-22
SEA
model0_SEA-22
SEA
model5_SEA-22
SEA
model6_SEA-22
SEA
model4_SEA-22
SEA
model1_SEA-22
SEA
model2_SEA-22
SEA
model7_SEA-22
SEA
SAM
rcp26_farfut
model11_SAM-22
NWS
NSA
SAM
NES
SWS
SES
SSA
model10_SAM-22
NWS
NSA
SAM
NES
SWS
SES
SSA
model6_SAM-22
NWS
NSA
SAM
NES
SWS
SES
SSA
model3_SAM-22
NWS
NSA
SAM
NES
SWS
SES
SSA
model1_SAM-22
NWS
NSA
SAM
NES
SWS
SES
SSA
mo

SAU
NZ
model11_AUS-22
NAU
CAU
EAU
SAU
NZ
model15_AUS-22
NAU
CAU
EAU
SAU
NZ
model9_AUS-22
NAU
CAU
EAU
SAU
NZ
model1_AUS-22
NAU
CAU
EAU
SAU
NZ
model6_AUS-22
NAU
CAU
EAU
SAU
NZ
model10_AUS-22
NAU
CAU
EAU
SAU
NZ
model0_AUS-22
NAU
CAU
EAU
SAU
NZ
rcp26_midfut
model4_AUS-22
NAU
CAU
EAU
SAU
NZ
model2_AUS-22
NAU
CAU
EAU
SAU
NZ
model1_AUS-22
NAU
CAU
EAU
SAU
NZ
model8_AUS-22
NAU
CAU
EAU
SAU
NZ
model5_AUS-22
NAU
CAU
EAU
SAU
NZ
model6_AUS-22
NAU
CAU
EAU
SAU
NZ
model3_AUS-22
NAU
CAU
EAU
SAU
NZ
model7_AUS-22
NAU
CAU
EAU
SAU
NZ
model0_AUS-22
NAU
CAU
EAU
SAU
NZ
rcp85_hist
model12_AUS-22
NAU
CAU
EAU
SAU
NZ
model17_AUS-22
NAU
CAU
EAU
SAU
NZ
model4_AUS-22
NAU
CAU
EAU
SAU
NZ
model16_AUS-22
NAU
CAU
EAU
SAU
NZ
model18_AUS-22
NAU
CAU
EAU
SAU
NZ
model19_AUS-22
NAU
CAU
EAU
SAU
NZ
model20_AUS-22
NAU
CAU
EAU
SAU
NZ
model13_AUS-22
NAU
CAU
EAU
SAU
NZ
model8_AUS-22
NAU
CAU
EAU
SAU
NZ
model5_AUS-22
NAU
CAU
EAU
SAU
NZ
model3_AUS-22
NAU
CAU
EAU
SAU
NZ
model7_AUS-22
NAU
CAU
EAU
SAU
NZ
model2_AUS-22
NAU
CAU
EAU
SAU
NZ
mo

SEAF
NEAF
ARP
model9_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model17_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model2_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model1_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model10_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model12_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
rcp85_midfut
model23_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model21_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model27_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model8_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model6_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model11_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model15_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model20_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model3_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model18_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model24_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEAF
NEAF
ARP
model13_AFR-22
WAF
SAH
CAF
WSAF
ESAF
MDG
SEA

model20_EUR-11
MED
WCE
NEU
model27_EUR-11
MED
WCE
NEU
model4_EUR-11
MED
WCE
NEU
model40_EUR-11
MED
WCE
NEU
model56_EUR-11
MED
WCE
NEU
model55_EUR-11
MED
WCE
NEU
model14_EUR-11
MED
WCE
NEU
model60_EUR-11
MED
WCE
NEU
model12_EUR-11
MED
WCE
NEU
model28_EUR-11
MED
WCE
NEU
model41_EUR-11
MED
WCE
NEU
model33_EUR-11
MED
WCE
NEU
model18_EUR-11
MED
WCE
NEU
model35_EUR-11
MED
WCE
NEU
model16_EUR-11
MED
WCE
NEU
model9_EUR-11
MED
WCE
NEU
model43_EUR-11
MED
WCE
NEU
model6_EUR-11
MED
WCE
NEU
model64_EUR-11
MED
WCE
NEU
model44_EUR-11
MED
WCE
NEU
model62_EUR-11
MED
WCE
NEU
model26_EUR-11
MED
WCE
NEU
model34_EUR-11
MED
WCE
NEU
model37_EUR-11
MED
WCE
NEU
rcp85_farfut
model5_EUR-11
MED
WCE
NEU
model30_EUR-11
MED
WCE
NEU
model53_EUR-11
MED
WCE
NEU
model61_EUR-11
MED
WCE
NEU
model25_EUR-11
MED
WCE
NEU
model52_EUR-11
MED
WCE
NEU
model21_EUR-11
MED
WCE
NEU
model63_EUR-11
MED
WCE
NEU
model59_EUR-11
MED
WCE
NEU
model50_EUR-11
MED
WCE
NEU
model42_EUR-11
MED
WCE
NEU
model57_EUR-11
MED
WCE
NEU
model39_EUR-11
MED


NWN
NEN
WNA
CNA
ENA
NCA
model19_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model15_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model8_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model17_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model12_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model1_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model16_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model11_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model4_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model18_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model3_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model6_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model14_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model9_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model7_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model0_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
GWL1.5
model9_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model12_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model1_rcp26_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model8_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model15_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model4_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model10_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NCA
model1_rcp85_NAM-22
NWN
NEN
WNA
CNA
ENA
NC

model4_rcp85_WAS-22
ARP
SAS
WCA
model9_rcp85_WAS-22
ARP
SAS
WCA
model0_rcp85_WAS-22
ARP
SAS
WCA
model2_rcp85_WAS-22
ARP
SAS
WCA
model24_rcp85_WAS-22
ARP
SAS
WCA
model1_rcp26_WAS-22
ARP
SAS
WCA
model6_rcp85_WAS-22
ARP
SAS
WCA
model20_rcp85_WAS-22
ARP
SAS
WCA
model10_rcp85_WAS-22
ARP
SAS
WCA
model11_rcp85_WAS-22
ARP
SAS
WCA
model4_rcp26_WAS-22
ARP
SAS
WCA
model23_rcp85_WAS-22
ARP
SAS
WCA
model21_rcp85_WAS-22
ARP
SAS
WCA
model17_rcp85_WAS-22
ARP
SAS
WCA
model15_rcp85_WAS-22
ARP
SAS
WCA
model25_rcp85_WAS-22
ARP
SAS
WCA
model2_rcp26_WAS-22
ARP
SAS
WCA
model5_rcp26_WAS-22
ARP
SAS
WCA
model16_rcp85_WAS-22
ARP
SAS
WCA
model14_rcp85_WAS-22
ARP
SAS
WCA
model13_rcp85_WAS-22
ARP
SAS
WCA
model6_rcp26_WAS-22
ARP
SAS
WCA
model8_rcp26_WAS-22
ARP
SAS
WCA
model9_rcp26_WAS-22
ARP
SAS
WCA
model7_rcp85_WAS-22
ARP
SAS
WCA
model8_rcp85_WAS-22
ARP
SAS
WCA
model3_rcp26_WAS-22
ARP
SAS
WCA
model22_rcp85_WAS-22
ARP
SAS
WCA
model12_rcp85_WAS-22
ARP
SAS
WCA
model19_rcp85_WAS-22
ARP
SAS
WCA
model1_rcp85_WAS-22
ARP
S

model3_EUR-11
MED
WCE
NEU
model65_EUR-11
MED
WCE
NEU
model17_EUR-11
MED
WCE
NEU
model23_EUR-11
MED
WCE
NEU
model20_EUR-11
MED
WCE
NEU
model27_EUR-11
MED
WCE
NEU
model4_EUR-11
MED
WCE
NEU
model40_EUR-11
MED
WCE
NEU
model56_EUR-11
MED
WCE
NEU
model55_EUR-11
MED
WCE
NEU
model14_EUR-11
MED
WCE
NEU
model60_EUR-11
MED
WCE
NEU
model12_EUR-11
MED
WCE
NEU
model28_EUR-11
MED
WCE
NEU
model41_EUR-11
MED
WCE
NEU
model33_EUR-11
MED
WCE
NEU
model18_EUR-11
MED
WCE
NEU
model35_EUR-11
MED
WCE
NEU
model16_EUR-11
MED
WCE
NEU
model9_EUR-11
MED
WCE
NEU
model43_EUR-11
MED
WCE
NEU
model6_EUR-11
MED
WCE
NEU
model44_EUR-11
MED
WCE
NEU
model62_EUR-11
MED
WCE
NEU
model26_EUR-11
MED
WCE
NEU
model34_EUR-11
MED
WCE
NEU
model37_EUR-11
MED
WCE
NEU
GWL1.5
model1_rcp26_EUR-11
MED
WCE
NEU
model61_rcp26_EUR-11
MED
WCE
NEU
model63_rcp26_EUR-11
MED
WCE
NEU
model58_rcp85_EUR-11
MED
WCE
NEU
model9_rcp85_EUR-11
MED
WCE
NEU
model55_rcp85_EUR-11
MED
WCE
NEU
model70_rcp85_EUR-11
MED
WCE
NEU
model41_rcp85_EUR-11
MED
WCE
NEU
model5

In [93]:
float(tmp.sel(region=list(tmp.abbrevs).index(region_name)).values)

5.350215387612606

In [15]:
ens_exp_dict_CORDEX['SEA'].keys()# ens_GWL_dict_CORDEX

['rcp85_1995-2014',
 'rcp26_2041-2060',
 'rcp85_2041-2060',
 'rcp26_2081-2100',
 'rcp85_2081-2100']

In [10]:
quantiles_dict = dict()
for short_CORDEX_domain in regional_averages_CORDEX:
    for clim_period in regional_averages_CORDEX[short_CORDEX_domain]:
        if clim_period not in quantiles_dict:
            quantiles_dict[clim_period] = dict()
        for region_name in regional_averages_CORDEX[short_CORDEX_domain][clim_period]:
            print clim_period, region_name
            #quantiles_dict[clim_period][region_name] = dict()
            dat = np.array(regional_averages_CORDEX[short_CORDEX_domain][clim_period][region_name])
            wdat = dat[~np.isnan(dat)]
            q10 = np.quantile(wdat, 0.1)
            q50 = np.quantile(wdat, 0.5)
            q90 = np.quantile(wdat, 0.9)
            quantiles_dict[clim_period][region_name] = [q10, q50, q90]

rcp85_farfut SEA
GWL4.0 SEA
GWL2.0 SEA
GWL1.5 SEA
rcp26_midfut SEA
rcp26_farfut SEA
rcp85_midfut SEA
rcp85_hist SEA
rcp85_farfut SES
rcp85_farfut SAM
rcp85_farfut NWS
rcp85_farfut NSA
rcp85_farfut SWS
rcp85_farfut SSA
rcp85_farfut NES
GWL4.0 SES
GWL4.0 SAM
GWL4.0 NWS
GWL4.0 NSA
GWL4.0 SWS
GWL4.0 SSA
GWL4.0 NES
GWL2.0 SES
GWL2.0 SAM
GWL2.0 NWS
GWL2.0 NSA
GWL2.0 SWS
GWL2.0 SSA
GWL2.0 NES
GWL1.5 SES
GWL1.5 SAM
GWL1.5 NWS
GWL1.5 NSA
GWL1.5 SWS
GWL1.5 SSA
GWL1.5 NES
rcp26_midfut SES
rcp26_midfut SAM
rcp26_midfut NWS
rcp26_midfut NSA
rcp26_midfut SWS
rcp26_midfut SSA
rcp26_midfut NES
rcp26_farfut SES
rcp26_farfut SAM
rcp26_farfut NWS
rcp26_farfut NSA
rcp26_farfut SWS
rcp26_farfut SSA
rcp26_farfut NES
rcp85_midfut SES
rcp85_midfut SAM
rcp85_midfut NWS
rcp85_midfut NSA
rcp85_midfut SWS
rcp85_midfut SSA
rcp85_midfut NES
rcp85_hist SES
rcp85_hist SAM
rcp85_hist NWS
rcp85_hist NSA
rcp85_hist SWS
rcp85_hist SSA
rcp85_hist NES
rcp85_farfut NEN
rcp85_farfut ENA
rcp85_farfut WNA
rcp85_farfut CNA
rcp8

In [11]:
quantiles_dict

{'GWL1.5': {'ARP': [2.959965460064001, 3.9387819861184274, 5.220115160493945],
  'CAF': [2.961756910466073, 3.9990012820058176, 4.838113469183649],
  'CAR': [3.358113186908499, 4.625694821260385, 5.859589466076846],
  'CAU': [3.9990664761761776, 4.958005176310635, 5.924515241018975],
  'CNA': [3.07086463548978, 3.5871365609896317, 4.4018190672565165],
  'EAS': [3.309198991511235, 3.9138983406511754, 4.491999908857152],
  'EAU': [4.236460739325334, 5.201491642416409, 6.304529192783107],
  'ECA': [2.6208968409928177, 3.245834824708305, 3.7279956945969266],
  'ENA': [2.7535906469412783, 3.2024838306151144, 3.7430939755730934],
  'ESAF': [3.9974731998399466, 4.913168064899987, 5.806233852793988],
  'MDG': [3.709108314542041, 4.569335592681724, 5.360533162781335],
  'MED': [3.8833319789919125, 4.2998108993847834, 5.041434799826911],
  'NAU': [4.26187063926646, 4.850072344576357, 5.555834480082142],
  'NCA': [3.6617548702842626, 4.584882929858316, 5.2180840534621895],
  'NEAF': [2.9811631822

In [12]:
import json
ensemble = 'CORDEX'
outfilename = '/home/jservon/Chapter12_IPCC/data/Figure_S12.3/'+ensemble+'_DF6_AR6_regional_averages.json'
print outfilename
with open(outfilename, 'w') as fp:
    json.dump(quantiles_dict, fp, sort_keys=True, indent=4)

/home/jservon/Chapter12_IPCC/data/Figure_S12.3/CORDEX_DF6_AR6_regional_averages.json


In [ ]:
# Creer une grille lon/lat pour chaque CORDEX domain
# regriller sur cette grille
# missing values outside the domain
#   => ok to retrieve the regions
#   => not ok to check if the region is completely covered
#   => check by hand?
#   => if CORDEX_domain = ... ; then subregions = [...]
#   => if partially covered => No!
# AFR, 3 domains for America, Aus = ok, 3 domains over Asia, 1 over Europe =
#   8 domains = feasable
# AFR, et Asia = ok; need Europe, and the Americas

In [ ]:
# Prendre un champ de chaque domain, faire un plot avec toutes les regions
# Selectionner a la main

# Si on prend les regions partiellement couvertes:
#  - on regrille CMIP6 et CMIP5 sur tous les domains CORDEX
#  - en multipliant par un mask pour mettre des missing 

In [11]:
ncdump(ens_CORDEX[short_CORDEX_domain][ens_exp][mem])

netcdf \074711fb002bc1317c7f5f1c4061cc2ab06afbc9ecb09c9df8dd67 {
dimensions:
	time = UNLIMITED ; // (1 currently)
	bnds = 2 ;
	x = 477 ;
	y = 497 ;
variables:
	double time(time) ;
		time:standard_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 2045-12-30" ;
		time:calendar = "360_day" ;
		time:axis = "T" ;
	double time_bnds(time, bnds) ;
	double lon(y, x) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "Longitude on Cross Points" ;
		lon:units = "degrees_east" ;
		lon:_CoordinateAxisType = "Lon" ;
	double lat(y, x) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "Latitude on Cross Points" ;
		lat:units = "degrees_north" ;
		lat:_CoordinateAxisType = "Lat" ;
	double x(x) ;
		x:standard_name = "projection_x_coordinate" ;
		x:long_name = "x-coordinate in Cartesian system" ;
		x:units = "m" ;
		x:axis = "X" ;
	double y(y) ;
		y:standard_name = "projection_y_coordinate" ;
		y:long_name = "y-coordinate in Cartesian system" ;
		y:units = "m" ;
		y:axis 

ncdump(time_average(ds('wbgt_cordex_ch12_test%%WBGTindoor%fx%global%rcp85%3.0K%AFR-22%MOHC-HadGEM2-ES--ICTP-RegCM4-7_r1i1p1%Exceed33')))

In [ ]:
# If region is not entireley covered => 0
# Do the map with regions on top and hatching
# Start the barplot:
# - retrieve values from json file
# - do the plot for one region
# Start to build the multiplot
# 

#  
#  
#  

In [26]:
regionmask.defined_regions.ar6.all.abbrevs.index('CAR')

8

In [12]:
ar6_land = regionmask.defined_regions.ar6.land
ar6_land

<regionmask.Regions>
Name:     AR6 reference regions (land only)
Source:   Iturbide et al., 2020 (Earth Syst. Sci. Data)

Regions:
  0  GIC  Greenland/Iceland
  1  NWN  N.W.North-America
  2  NEN  N.E.North-America
  3  WNA    W.North-America
  4  CNA    C.North-America
..   ...                ...
 41  EAU        E.Australia
 42  SAU        S.Australia
 43   NZ        New-Zealand
 44  EAN       E.Antarctica
 45  WAN       W.Antarctica

[46 regions]

In [13]:
ar6_land.abbrevs

[u'GIC',
 u'NWN',
 u'NEN',
 u'WNA',
 u'CNA',
 u'ENA',
 u'NCA',
 u'SCA',
 u'CAR',
 u'NWS',
 u'NSA',
 u'NES',
 u'SAM',
 u'SWS',
 u'SES',
 u'SSA',
 u'NEU',
 u'WCE',
 u'EEU',
 u'MED',
 u'SAH',
 u'WAF',
 u'CAF',
 u'NEAF',
 u'SEAF',
 u'WSAF',
 u'ESAF',
 u'MDG',
 u'RAR',
 u'WSB',
 u'ESB',
 u'RFE',
 u'WCA',
 u'ECA',
 u'TIB',
 u'EAS',
 u'ARP',
 u'SAS',
 u'SEA',
 u'NAU',
 u'CAU',
 u'EAU',
 u'SAU',
 u'NZ',
 u'EAN',
 u'WAN']

In [14]:
ar6_land.names

[u'Greenland/Iceland',
 u'N.W.North-America',
 u'N.E.North-America',
 u'W.North-America',
 u'C.North-America',
 u'E.North-America',
 u'N.Central-America',
 u'S.Central-America',
 u'Caribbean',
 u'N.W.South-America',
 u'N.South-America',
 u'N.E.South-America',
 u'South-American-Monsoon',
 u'S.W.South-America',
 u'S.E.South-America',
 u'S.South-America',
 u'N.Europe',
 u'West&Central-Europe',
 u'E.Europe',
 u'Mediterranean',
 u'Sahara',
 u'Western-Africa',
 u'Central-Africa',
 u'N.Eastern-Africa',
 u'S.Eastern-Africa',
 u'W.Southern-Africa',
 u'E.Southern-Africa',
 u'Madagascar',
 u'Russian-Arctic',
 u'W.Siberia',
 u'E.Siberia',
 u'Russian-Far-East',
 u'W.C.Asia',
 u'E.C.Asia',
 u'Tibetan-Plateau',
 u'E.Asia',
 u'Arabian-Peninsula',
 u'S.Asia',
 u'S.E.Asia',
 u'N.Australia',
 u'C.Australia',
 u'E.Australia',
 u'S.Australia',
 u'New-Zealand',
 u'E.Antarctica',
 u'W.Antarctica']